In [1]:
import mysql.connector
import pandas as pd
import time
import os

In [2]:
# 從 cursor印出以 dataframe呈現的內容(預設顯示頭 15筆)
def showdata(cursor, header=15):
    df = pd.DataFrame(cursor.fetchall())
    try:
        df.columns = cursor.column_names
        if header == 'all':
            header = len(df)
    except:
        pass
    
    finally:
        display(df.head(header))

In [3]:
# 連結 MySQL
conn = mysql.connector.connect(host = "localhost",
                               user = "root",
                               password = "yourpassword")
cursor = conn.cursor()

# 初始化
cursor.execute("CREATE DATABASE IF NOT EXISTS mydb")

# 字元集、字元序及儲存引擎

## 字元集(CHARACTER SET)
電腦在處理文字資料大多是使用一個「編碼」來表示某一個字，對MySQL資料庫來說，為了要處理不同語言的文字，它使用一套編碼來處理一種語言的文字，稱為「字元集」(CHARACTER SET)。

In [4]:
# 查詢 MySQL 資料庫支援的字元集資訊
cursor.execute("SHOW CHARACTER SET")
showdata(cursor, 5)

,Charset,Description,Default collation,Maxlen
0,big5,Big5 Traditional Chinese,big5_chinese_ci,2
1,dec8,DEC West European,dec8_swedish_ci,1
2,cp850,DOS West European,cp850_general_ci,1
3,hp8,HP West European,hp8_english_ci,1
4,koi8r,KOI8-R Relcom Russian,koi8r_general_ci,1


## 字元序(COLLATION)
字元序指的是在一個字元集中，所有字元的大小排序規則。有時候我們不需要區分大小寫，以英文字母來說，將A和a視為相同，在這種情況下則稱為「Case-insensitive Collation (ci)」，因此也有區分大小寫的「Case-sensitive Collation (cs)」，通常是為了各地區的字符排序需求而使用。此外還有常見的二進制型態，稱為「Binary Collation (bin)」，意思是只要字符不同(ex:大小寫或變音符號)，則完全視為不同，優點視查詢速度極有效率，而缺點是排序順序不自然。

* **_bin**：按字符串中每個字符的二進制值比較字符串。
* **_ci**：使用通用語言規則和不區分大小寫的比較來比較字符串。
* **_cs**：使用通用語言規則和區分大小寫的比較來比較字符串。


In [5]:
# 查詢 MySQL 支援的字元序
cursor.execute("SHOW COLLATION")
showdata(cursor, 5)

,Collation,Charset,Id,Default,Compiled,Sortlen
0,big5_chinese_ci,big5,1,Yes,Yes,1
1,big5_bin,big5,84,,Yes,1
2,dec8_swedish_ci,dec8,3,Yes,Yes,1
3,dec8_bin,dec8,69,,Yes,1
4,cp850_general_ci,cp850,4,Yes,Yes,1


## 儲存引擎(ENGINE)
「Storage engine」是MySQL用來儲存資料的技術，為了資料庫多樣化的應用，使用者可以在建立表格的時候，依照自己的需求指定一種儲存引擎，不同的儲存引擎會有不同的資料儲存方式與運作的特色。MySQL提供許多儲存引擎讓使用者選擇，下列是主要的三種儲存引擎的簡介：  

   + **MyISAM**：支援的功能並沒有像一般的資料庫那麼多(例如交易)；不過也因為它比較簡單，所以運作的效率相對也比較好。  
   + **InnoDB**：這種儲存引擎所提供的功能已經跟大型的商用資料庫軟體一樣了，像是交易、紀錄鎖定與自動回復。  
   + **MEMORY**：把資料儲存在紀憶體中，所以運作的效率是最快的；不過只要MySQL伺服器關閉後，儲存的資料就全部不見了。

In [6]:
# 顯示所支援的全部引擎
cursor.execute("SHOW ENGINES")
showdata(cursor)

,Engine,Support,Comment,Transactions,XA,Savepoints
0,InnoDB,DEFAULT,"Supports transactions, row-level locking, and ...",YES,YES,YES
1,MRG_MYISAM,YES,Collection of identical MyISAM tables,NO,NO,NO
2,MEMORY,YES,"Hash based, stored in memory, useful for tempo...",NO,NO,NO
3,BLACKHOLE,YES,/dev/null storage engine (anything you write t...,NO,NO,NO
4,MyISAM,YES,MyISAM storage engine,NO,NO,NO
5,CSV,YES,CSV storage engine,NO,NO,NO
6,ARCHIVE,YES,Archive storage engine,NO,NO,NO
7,PERFORMANCE_SCHEMA,YES,Performance Schema,NO,NO,NO
8,FEDERATED,NO,Federated MySQL storage engine,None,None,None


# 資料庫

## 刪除資料庫(DROP DATABASE)
如果在刪除資料庫的敘述中指定的資料庫名稱不存在，MySQL會產生一個錯誤訊息，為了避免上列的錯誤，使用者可以在刪除資料庫的時候，加入「IF EXISTS」指令，如果使用者指定的資料庫名稱存在，同樣會刪除指定的資料庫；如果不存在，MySQL只會產生警告訊息。
  
註：執行刪除資料庫的敘述，MySQL不會再跟使用者確認是否刪除資料庫，而是直接刪除；刪除資料庫以後，表示資料庫資料夾也會從檔案系統中刪除，除非使用者另外還有這個資料庫的備份，否則原來在資料庫中的所有資料就全部消失了。

In [7]:
# 刪除資料庫前現有資料庫
print("刪除資料庫前:")
cursor.execute("SHOW DATABASES")
showdata(cursor)

# 刪除資料庫
cursor.execute("DROP DATABASE IF EXISTS mydb")

# 刪除資料庫後現有資料庫
print("刪除資料庫後:")
cursor.execute("SHOW DATABASES")
showdata(cursor)

刪除資料庫前:


,Database
0,information_schema
1,cmdev
2,mydb
3,mysql
4,performance_schema
5,sakila
6,scraping
7,sys
8,world


刪除資料庫後:


,Database
0,information_schema
1,cmdev
2,mysql
3,performance_schema
4,sakila
5,scraping
6,sys
7,world


## 建立資料庫(CREATE DATABASE)
在使用者執行新增資料庫的指令以後，MySQL會使用使用者指定的資料庫名稱建立一個資料庫資料夾，如果使用者指定的資料庫名稱已經存在了，MySQL會產生一個錯誤訊息，為了避免錯誤，使用者可以在建立資料庫的時候加入「IF NOT EXISTS」，如果使用者指定的資料庫名稱不存在，同樣會建立新的資料庫；如果已經存在，MySQL只會產生警告訊息。
  
使用者也可以在建立資料庫的時候指定資料庫預設的字元集與字元序，如果沒有指定的話，就會使用MySQL伺服器預設的設定:字元集是「utf8」，字元序是「utf8_general_ci」。

In [8]:
# 建立資料庫前現有資料庫
print("建立資料庫前:")
cursor.execute("SHOW DATABASES")
showdata(cursor)

# 建立資料庫並設定字元集為 big5而字元序為 big5_chinese_ci
cursor.execute("CREATE DATABASE IF NOT EXISTS mydb CHARACTER SET big5 COLLATE big5_chinese_ci")

# 建立資料庫後現有資料庫
print("建立資料庫後:")
cursor.execute("SHOW DATABASES")
showdata(cursor)

建立資料庫前:


,Database
0,information_schema
1,cmdev
2,mysql
3,performance_schema
4,sakila
5,scraping
6,sys
7,world


建立資料庫後:


,Database
0,information_schema
1,cmdev
2,mydb
3,mysql
4,performance_schema
5,sakila
6,scraping
7,sys
8,world


##  修改資料庫(ALTER DATABASE)
建立資料庫以後，使用者唯一能執行的修改是資料庫預設的字元集與字元序。

In [9]:
# 修改資料庫前的字元集與字元序
print("修改資料庫前:")
cursor.execute("SELECT * FROM information_schema.SCHEMATA WHERE SCHEMA_NAME = 'mydb'")
showdata(cursor)

# 修改資料庫預設的字元集與字元序
cursor.execute("ALTER DATABASE mydb CHARACTER SET utf8 COLLATE utf8_general_ci")

# 修改資料庫後的字元集與字元序
print("修改資料庫後:")
cursor.execute("SELECT * FROM information_schema.SCHEMATA WHERE SCHEMA_NAME = 'mydb'")
showdata(cursor)

修改資料庫前:


,CATALOG_NAME,SCHEMA_NAME,DEFAULT_CHARACTER_SET_NAME,DEFAULT_COLLATION_NAME,SQL_PATH
0,def,mydb,big5,big5_chinese_ci,None


修改資料庫後:


,CATALOG_NAME,SCHEMA_NAME,DEFAULT_CHARACTER_SET_NAME,DEFAULT_COLLATION_NAME,SQL_PATH
0,def,mydb,utf8,utf8_general_ci,None


# 表格

## 建立表格(CREATE TABLE)
MySQL規定一個表格中至少要有一個欄位，在設定表格中的欄位時，至少要明確的決定欄位的名稱與型態，其它的欄位設定都是選擇性的，如果有一個以上欄位，要使用逗號隔開，且建立表格的時候可以使用「IF NOT EXISTS」選項，預防發生表格已存在的錯誤。

In [10]:
# 切換目前使用資料庫
cursor.execute("USE mydb")

# 建立表格前資料庫內所有表格
print("建立表格前:")
cursor.execute("SHOW TABLES")
showdata(cursor)

# 建立表格
cursor.execute("CREATE TABLE IF NOT EXISTS addressbook( \
                name       VARCHAR(20), \
                age        TINYINT(3), \
                tel        VARCHAR(10), \
                email      VARCHAR(80), \
                birthdate  DATE)")

# 建立表格後資料庫內所有表格
print("建立表格後:")
cursor.execute("SHOW TABLES")
showdata(cursor)

建立表格前:


""


建立表格後:


,Tables_in_mydb
0,addressbook


### 使用其它表格建立一個新表格
在一些比較特別的情況會使用一個現有的表格來建立新的表格，而建立的新表格，可以省略欄位定義的工作，新表格會使用原有表格的欄位名稱與定義，而且在查詢敘述中傳回的資料會直接新增到新建立的表格中，不過，也可以在建立新表格的時候，使用欄位定義來設定新表格的欄位型態與其它屬性，或是加入查詢敘述中沒有的欄位。

註：如果沒有指定儲存引擎、字元集或字元序的話，建立的新表格使用資料庫的預設。查詢表格中，欄位的索引與「AUTO_INCREMENT」設定都會被忽略。

In [11]:
# 使用查詢表格來建立新表格並加入新欄位 tel
cursor.execute("CREATE TABLE IF NOT EXISTS addressbook_transfer( \
                empno   VARCHAR(4) NOT NULL, \
                ename   VARCHAR(20) CHARACTER SET utf8 NOT NULL, \
                job     VARCHAR(20), \
                tel     VARCHAR(10)) \
                SELECT empno, ename, job \
                FROM cmdev.emp")

# 顯示指定表格欄位資訊
cursor.execute("SELECT * FROM addressbook_transfer")
showdata(cursor, 5)

,tel,empno,ename,job
0,None,7369,SMITH,CLERK
1,None,7499,ALLEN,SALESMAN
2,None,7521,WARD,SALESMAN
3,None,7566,JONES,MANAGER
4,None,7654,MARTIN,SALESMAN


## 修改表格名稱(RENAME TABLE)
如果需要修改表格的名稱，可以使用「RENAME TABLE」敘述。

In [12]:
# 顯示更改表格名前資料庫內所有表格資訊
print("更改表格名前:")
cursor.execute("SHOW TABLES")
showdata(cursor)

# 更改表格名
cursor.execute("RENAME TABLE addressbook TO addressbook_rename")

# 顯示更改表格名後資料庫內所有表格資訊
print("更改表格名後:")
cursor.execute("SHOW TABLES")
showdata(cursor)

更改表格名前:


,Tables_in_mydb
0,addressbook
1,addressbook_transfer


更改表格名後:


,Tables_in_mydb
0,addressbook_rename
1,addressbook_transfer


## 表格屬性(ENGINE, CHARACTER SET, COLLATE)
在建立表格的時侯可以針對表格的需求，設定它使用的儲存引擎、字元集與字元序，如果沒有指定這些屬性，MySQL會使用伺服器預設的儲存引擎(InnoDB)作為表格的儲存引擎，字元集(utf8)與字元序(utf8_general_ci)會使用資料庫預設的設定。

In [13]:
# 建立表格，並使用預設設定
cursor.execute("CREATE TABLE IF NOT EXISTS addressbook_default( \
                name       VARCHAR(20), \
                age        TINYINT(3))")

# 建立表格，並針對表格的需求，設定它使用的儲存引擎、字元集與字元序
cursor.execute("CREATE TABLE IF NOT EXISTS addressbook_customize( \
                name       VARCHAR(20), \
                age        TINYINT(3)) \
                ENGINE = MyISAM \
                CHARACTER SET = big5 \
                COLLATE = big5_bin")

# 顯示資料庫內所有表格資訊
cursor.execute("SHOW TABLE STATUS")
showdata(cursor)

,Name,Engine,Version,Row_format,Rows,Avg_row_length,Data_length,Max_data_length,Index_length,Data_free,Auto_increment,Create_time,Update_time,Check_time,Collation,Checksum,Create_options,Comment
0,addressbook_customize,MyISAM,10,Dynamic,0,0,0,281474976710655,1024,0,None,2020-03-29 16:50:04,2020-03-29 16:50:04,None,big5_bin,None,,
1,addressbook_default,InnoDB,10,Dynamic,0,0,16384,0,0,0,None,2020-03-29 16:50:03,NaT,None,utf8_general_ci,None,,
2,addressbook_rename,InnoDB,10,Dynamic,0,0,16384,0,0,0,None,2020-03-29 16:50:01,NaT,None,utf8_general_ci,None,,
3,addressbook_transfer,InnoDB,10,Dynamic,14,1170,16384,0,0,0,None,2020-03-29 16:50:01,2020-03-29 16:50:02,None,utf8_general_ci,None,,


## 刪除表格(DROP TABLE)
使用「DROP TABLE」敘述執行刪除表格的工作時，MySQL並不會再次跟使用者確認是否真的要刪除，而是真的就直接刪除了，表格儲存的紀錄資料當然也不見了。

In [14]:
# 刪除表格前資料庫內所有表格
print("刪除表格前:")
cursor.execute("SHOW TABLES")
showdata(cursor)

# 刪除所有表格
cursor.execute("DROP TABLE IF EXISTS addressbook_rename")
cursor.execute("DROP TABLE IF EXISTS addressbook_transfer")
cursor.execute("DROP TABLE IF EXISTS addressbook_default")
cursor.execute("DROP TABLE IF EXISTS addressbook_customize")

# 刪除表格後資料庫內所有表格
print("刪除表格後:")
cursor.execute("SHOW TABLES")
showdata(cursor)

刪除表格前:


,Tables_in_mydb
0,addressbook_customize
1,addressbook_default
2,addressbook_rename
3,addressbook_transfer


刪除表格後:


""


## 欄位屬性
在建立表格時，使用者會幫每一個欄位指定適合的「資料型態、data type」。正確的選擇欄位資料型態，除了可以幫使用者儲存正確的資料外，還可以讓資料庫使用最少的記憶體與儲存空間，這樣會讓資料庫運作的效率更好一些。

### 字串欄位(VARCHAR, ENUM, SET)  
包含**非二進位制**與**二進位制**兩種字串值，非二進位制字串值是一些使用字元集與字元序的**字元(character)**組合起來的；二進位制字串值是一些**位元組(bytes)**組合的資料。如果一個欄位的型態是字串的話，使用者還可以依照需求加入字串型態的欄位屬性。  

**非二進位制**就是**儲存一般文字**的字串，會有特定的字元集與字元序，所以可以用來儲存各種不同國家的文字，且不同的字元集會佔用不同的儲存空間。

| TYPE          | MAX LENGHT | PROPERTY        |
|---------------|------------|-----------------|
| CHAR(len.)    | 255        | Fixed length    |
| VARCHAR(len.) | 65535      | Variable length |
| TINYTEXT      | 255        | Variable length |
| TEXT          | 65535      | Variable length |
| MEDIUMTEXT    | 16772215   | Variable length |
| LONGTEXT      | 4294967295 | Variable length |

| CHARATER SET | MAX BYTE(S) FOR A CHARATER |
|--------------|----------------------------|
| latin1       |              1             |
| big5         |              2             |
| utf8         |              3             |


**二進位制**的字串型態是使用位元組(byte)為單位來儲存字串資料，跟非二進位制的字串類似，且所有二進位制的字串型態都**不可以**指定字元集與字元序，不過也可以使用它們來儲存任何語言的文字，或是**儲存類似音樂或圖片資料**。

| TYPE            | MAX LENGHT | PROPERTY        |
|-----------------|------------|-----------------|
| BINARY(len.)    | 255        | Fixed length    |
| VARBINARY(len.) | 65535      | Variable length |
| TINYBLOB        | 255        | Variable length |
| BLOB            | 65535      | Variable length |
| MEDIUMBLOB      | 16772215   | Variable length |
| LONGBLOB        | 4294967295 | Variable length |

**列舉(ENUM)**與**集合(SET)**是一種特殊的非二進位制字串型態，所以它們也可以指定字元集與字元序，而列舉型態能限定儲存列舉內的**單一元素**，集合型態和列舉類似，但最大的不同是集合型態能儲存集合內**任意的元素組合**，此外，這兩種資料型態也可以有順序，其順序為建立表格時由使用者設定的順序。

| TYPE                 | MAX NO. OF ELEMENTS |
|----------------------|---------------------|
| ENUM(str1, str2,...) | 65535               |
| SET(str1, str2,...)  | 64                  |

In [15]:
# 建立表格
cursor.execute("CREATE TABLE IF NOT EXISTS cloth( \
                order_name    VARCHAR(30) CHARACTER SET big5 COLLATE big5_chinese_ci, \
                order_time    ENUM('MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN') CHARACTER SET utf8 COLLATE utf8_general_ci, \
                order_size    SET('XS', 'S', 'M', 'L', 'XL') CHARACTER SET utf8 COLLATE utf8_general_ci)")

# 新增資料到表格中，注意 order_size欄位為 SET 型態，鍵入資料時不能有空白(正確: 'XS,M,L'; 錯誤: 'XS, M, L')
cursor.execute("INSERT INTO cloth VALUES ('Mary', 'TUE', 'XS,M,L'), \
                                         ('Cathy', 'SUN', 'XS,XS'), \
                                         ('Lin', 'FRI', 'XL')")

# 顯示表格欄位內容
print("cloth:")
cursor.execute("SELECT * FROM cloth")
showdata(cursor)

# 顯示指定表格欄位資訊
cursor.execute("SHOW FULL COLUMNS FROM cloth")
showdata(cursor)

cloth:


,order_name,order_time,order_size
0,Mary,TUE,"{XS, M, L}"
1,Cathy,SUN,{XS}
2,Lin,FRI,{XL}


,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,order_name,varchar(30),big5_chinese_ci,YES,,None,,"select,insert,update,references",
1,order_time,"enum('MON','TUE','WED','THU','FRI','SAT','SUN')",utf8_general_ci,YES,,None,,"select,insert,update,references",
2,order_size,"set('XS','S','M','L','XL')",utf8_general_ci,YES,,None,,"select,insert,update,references",


### 數值欄位(INT, FLOAT, UNSIGNED, ZEROFILL, AUTO_INCREMENT)  
任何包含**正、負號的整數與小數資料**；另外還有**位元(bit)的數值資料**，它使用二進位來表示一個數字。  

**整數**型態的意思就是它們不能儲存小數，在建立表格的時候，如果需要一個可以儲存整數資料的欄位，使用者可以依照整數資料的大小需求，選擇一個夠用又不會太浪費空間的整數形態。

| TYPE            | BYTE(s) | DEFALUT LEN. | SIGNED RANGE                               | UNSIGNED RANGE           |
|-----------------|---------|--------------|--------------------------------------------|--------------------------|
| TINYINT(len.)   |    1    |       4      | -128 ~ 127                                 | 0 ~ 255                  |
| SMALLINT(len.)  |    2    |       6      | -32768 ~ 32767                             | 0 ~ 65535                |
| MEDIUMINT(len.) |    3    |       9      | -8388608 ~ 8388607                         | 0 ~ 16777215             |
| INT(len.)       |    4    |      11      | -2147683648 ~ 2147683647                   | 0 ~ 4294967295           |
| BIGINT(len.)    |    8    |      20      | -9223372036854775808 ~ 9223372036854775807 | 0 ~ 18446744073709551615 |


**「FLOAT」**和**「DOUBLE」**型態的欄位可以用來儲存包含**小數**的數值，儲存空間分別是4和8個位元組，它們是一種佔用儲存空間比較小，執行運算比較快的型態。不過因為它們是使用**「近似值」**來儲存使用者的數值，所以如果使用者需要儲存完全精準的數值，就不能使用這兩種型態。另外一種可以儲存小數數值的**「DECIMAL」**型態就可以用來儲存**完全精準**的數值，儲存在這個型態中的數值，不論是查詢或是運算，都不會有任何誤差，不過「DECIMAL」型態佔用的儲存空間就比「FLOAT」和「DOUBLE」型態大。  

| TYPE                       | BYTE(s)         | DEFALUT LEN.    |
|----------------------------|-----------------|-----------------|
| FLOAT(len.,decimal_len.)   | 4               | depends on sys. |
| DOUBLE(len.,decimal_len.)  | 8               | depends on sys. |
| DECIMAL(len.,decimal_len.) | depends on user | 10,0            | 


數值型態欄位專用的屬性設定有**「UNSIGNED」**、**「ZEROFILL」**與**「AUTO_INCREMENT」**。
   + **UNSIGNED**：只能儲存**正數**。
   + **ZEROFILL**：在查詢這些欄位的時候，回傳的資料會在左側根據長度的設定**填滿「0」**。
   + **AUTO_INCREMENT**：如果資料表中的每一筆紀錄需要**流水號**，可以選擇正整數型態的欄位後，再使用「AUTO_INCREMENT」欄位屬性並為它建立主索引鍵。設定為「AUTO_INCREMENT」的整數欄位，在新增資料的時候可以不用指定數值，MySQL會為使用者自動編製一個流水號並儲存在紀錄中；而要取得自動編製的流水號，可以使用**「LAST_INSERT_ID()」**函式。
      - 一個表格只能有**一個「AUTO_INCREMENT」欄位**，**而且要為它建立一個索引**，而且通常是建立主索引鍵或唯一索引，這樣可以防止重複的編號；不過MySQL也允許使用者建立可重複的索引
      - 只有**整數型態才可以使用「AUTO_INCREMENT」**欄位屬性，使用者可以根據編號大小的需求，選擇使用「TINYINT」、「SMALLINT」、「MEDIUMINT」、「INT」或「BIGINT」，而且因為只會使用到正數，所以使用者可以加入「UNSIGNED」來增加編號的範圍
      - 如果編號已經到欄位型態的最大範圍，例如一個「SMALLINT」型態，而且是指定為「UNSIGNED」的「AUTO_INCREMENT」欄位，編號已經到「65535」了，如果再執行新增的敘述，就會造成「Duplicate entry ’65535′ for key ‘欄位名稱’」的錯誤

In [16]:
# 建立表格(ZEROFILL 在進入 pandas時會自動被去掉 0)
cursor.execute("CREATE TABLE IF NOT EXISTS meeting( \
                id       SMALLINT(3) UNSIGNED ZEROFILL AUTO_INCREMENT, \
                title    VARCHAR(50), \
                duration FLOAT(4,2) UNSIGNED ZEROFILL, \
                PRIMARY KEY (id))")

cursor.execute("CREATE TABLE IF NOT EXISTS participate( \
                id       SMALLINT(3) UNSIGNED ZEROFILL AUTO_INCREMENT, \
                name     VARCHAR(20), \
                INDEX (id))")

meeting_info = [('Sales meeting', 1.52), ('Marketing meeting', 17.33)]
meeting_members = [('John', 'Amy', 'David'), ('Amy', 'Andy')]

for info, members in zip(meeting_info, meeting_members):
    # 新增資料到 meeting 表格中的 title 欄位, 不指定 id 欄位可以自動新增流水號
    cursor.execute("INSERT INTO meeting (title, duration) VALUES (%s, %s)", (info[0], info[1]))
    
    for member in members:
        # 使用 LAST_INSERT_ID() 函式來取得流水號(注意最後一個逗點的特殊用法)
        cursor.execute("INSERT INTO participate (id, name) VALUES (LAST_INSERT_ID(), %s)", (member,))

# 顯示指定表格欄位內容
print("meeting:")
cursor.execute("SELECT * FROM meeting")
showdata(cursor)
print("participate:")
cursor.execute("SELECT * FROM participate")
showdata(cursor)

# 使用 id 作為結合查詢條件
cursor.execute("SELECT meeting.id, meeting.title, meeting.duration, participate.name \
                FROM meeting, participate \
                WHERE meeting.id = participate.id")
showdata(cursor)

# 顯示指定表格欄位資訊
print("meeting:")
cursor.execute("SHOW FULL COLUMNS FROM meeting")
showdata(cursor)
print("participate:")
cursor.execute("SHOW FULL COLUMNS FROM participate")
showdata(cursor)

meeting:


,id,title,duration
0,1,Sales meeting,1.52
1,2,Marketing meeting,17.33


participate:


,id,name
0,1,John
1,1,Amy
2,1,David
3,2,Amy
4,2,Andy


,id,title,duration,name
0,1,Sales meeting,1.52,John
1,1,Sales meeting,1.52,Amy
2,1,Sales meeting,1.52,David
3,2,Marketing meeting,17.33,Amy
4,2,Marketing meeting,17.33,Andy


meeting:


,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,id,smallint(3) unsigned zerofill,None,NO,PRI,None,auto_increment,"select,insert,update,references",
1,title,varchar(50),utf8_general_ci,YES,,None,,"select,insert,update,references",
2,duration,"float(4,2) unsigned zerofill",None,YES,,None,,"select,insert,update,references",


participate:


,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,id,smallint(3) unsigned zerofill,None,NO,MUL,None,auto_increment,"select,insert,update,references",
1,name,varchar(20),utf8_general_ci,YES,,None,,"select,insert,update,references",


### 通用欄位(NULL, NOT NULL, DEFAULT)
除了字串與數值兩種欄位專用的欄位屬性設定外，還有許多可以用在所有型態的欄位屬性。  
   + **NOT NULL**：**不允許儲存「NULL」值**  
   + **NULL**：表示一個欄位的資料是「未知」或「沒有」，如果沒有使用「DEFAULT」設定預設值，MySQL會自動加入**預設「NULL」**的設定
   + **DEFAULT**：設定欄位的**預設值**，可以自己指定任何想要的預設值。

In [17]:
# 建立表格，並針對個欄位屬性做設定
cursor.execute("CREATE TABLE IF NOT EXISTS income( \
                name    VARCHAR(20) NOT NULL, \
                age     TINYINT(3) NULL, \
                salary  INT(10) DEFAULT 22000)")

# 新增資料
cursor.execute("INSERT INTO income VALUES ('Amy', 31, 37000)")
cursor.execute("INSERT INTO income (name, age) VALUES ('Andy', 19)")
cursor.execute("INSERT INTO income (name, salary) VALUES ('John', 50000)")

# 顯示指定表格欄位內容
print("income:")
cursor.execute("SELECT * FROM income")
showdata(cursor)

# 顯示指定表格欄位資訊
cursor.execute("SHOW FULL COLUMNS FROM income")
showdata(cursor)

income:


,name,age,salary
0,Amy,31.0,37000
1,Andy,19.0,22000
2,John,NaN,50000


,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,name,varchar(20),utf8_general_ci,NO,,NaN,,"select,insert,update,references",
1,age,tinyint(3),None,YES,,NaN,,"select,insert,update,references",
2,salary,int(10),None,YES,,22000.0,,"select,insert,update,references",


### 時間欄位  
時間的資料性質包含日期、時間、日期加時間、西元年及TIMESTAMP。

| TYPE         | BYTE(s) | RANGE                                         |
|--------------|---------|-----------------------------------------------|
| DATE         |    3    | 1000-01-01 ~ 9999-12-31                       |
| TIME         |    3    | -838:59:59 ~ 838:59:59                        |
| DATETIME     |    8    | 1000-01-01 00:00:00 ~ 9999-12-31 23:59:59     |
| YEAR(4 or 2) |    1    | 1901~2155 for YEAR(4) ~ 1970~2069 for YEAR(2) |
| TIMESTAMP    |    4    | 1970-01-01 00:00:00 ~ 2037                    |


在表格中使用「TIMESTAMP」型態的欄位時，如果使用者沒有設定它們的欄位屬性，MySQL會自動幫使用者在第一個「TIMESTAMP」欄位加入「NOT NULL」、「DEFAULT」和「ON UPDATE」三個欄位屬性的設定。其它沒有設定欄位屬性的「TIMESTAMP」欄位，MySQL會幫使用者加入「NOT NULL」與「DEFAULT」兩個欄位屬性。  
   + **NOT NULL**：不允許儲存「NULL」值
   + **DEFAULT CURRENT_TIMESTAMP**：設定預設值為目前的日期時間
   + **ON UPDATE**：在修改紀錄的時候，MySQL自動填入指定的資料  
  
可是如果在同一筆紀錄中，要使用一個欄位記錄新增資料的日期與時間，而使用另一個欄位記錄修改資料的日期與時間。為了應付這樣的需求，必須使用MySQL提供給使用者的特殊設定方式來解決，因為MySQL限制在一個表格中「CURRENT_TIMESTAMP」只能在一個欄位出現

In [18]:
# 建立表格，並針對個欄位屬性做設定
cursor.execute("CREATE TABLE IF NOT EXISTS birthday( \
                name          VARCHAR(20) NOT NULL, \
                birthdate     DATE DEFAULT NULL, \
                registertime  TIMESTAMP DEFAULT 0, \
                updatetime    TIMESTAMP ON UPDATE CURRENT_TIMESTAMP)")

# 顯示指定表格欄位資訊
cursor.execute("SHOW FULL COLUMNS FROM birthday")
showdata(cursor)

# 利用 INSERT 新增資料時，只會在 registertime 欄位新增寫入時間
cursor.execute("INSERT INTO mydb.birthday VALUES \
               ('SMITH',DEFAULT,NULL,DEFAULT), \
               ('ALLEN','2000-10-02',NULL,DEFAULT), \
               ('WARD','1963-01-11',NULL,DEFAULT), \
               ('JONES','2016-07-04',NULL,DEFAULT), \
               ('MARTIN','1992-04-30',NULL,DEFAULT)")

# 顯示指定表格欄位內容
print("新增紀錄的時更新時間:")
cursor.execute("SELECT * FROM birthday")
showdata(cursor)

time.sleep(5)

# 利用 UPDATE 修改資料時，會在 updatetime欄位更新時間
cursor.execute("UPDATE IGNORE mydb.birthday \
                SET birthdate = '1987-02-20' \
                WHERE name = 'SMITH'")

# 顯示指定表格欄位內容
print("修改紀錄的時更新時間:")
cursor.execute("SELECT * FROM birthday")
showdata(cursor)

,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,name,varchar(20),utf8_general_ci,NO,,None,,"select,insert,update,references",
1,birthdate,date,None,YES,,None,,"select,insert,update,references",
2,registertime,timestamp,None,NO,,0000-00-00 00:00:00,,"select,insert,update,references",
3,updatetime,timestamp,None,NO,,0000-00-00 00:00:00,on update CURRENT_TIMESTAMP,"select,insert,update,references",


新增紀錄的時更新時間:


,name,birthdate,registertime,updatetime
0,SMITH,None,2020-03-29 16:50:12,None
1,ALLEN,2000-10-02,2020-03-29 16:50:12,None
2,WARD,1963-01-11,2020-03-29 16:50:12,None
3,JONES,2016-07-04,2020-03-29 16:50:12,None
4,MARTIN,1992-04-30,2020-03-29 16:50:12,None


修改紀錄的時更新時間:


,name,birthdate,registertime,updatetime
0,SMITH,1987-02-20,2020-03-29 16:50:12,2020-03-29 16:50:17
1,ALLEN,2000-10-02,2020-03-29 16:50:12,NaT
2,WARD,1963-01-11,2020-03-29 16:50:12,NaT
3,JONES,2016-07-04,2020-03-29 16:50:12,NaT
4,MARTIN,1992-04-30,2020-03-29 16:50:12,NaT


## 修改欄位(ALTER TABLE)
建立表格以後，如果發現某個欄位或設定打錯，或是在使用一陣子以後，發覺表格中有一些設定不太對，可以使用「ALTER TABLE」敘述來修改一個表格的結構

### 增加欄位(ADD)
用「ADD」來增加一個本來沒有的欄位，而如果在增加欄位的時候，沒有指定新增欄位的位置，MySQL會把這個欄位放在最後一個，若要調整欄位的位置則需搭配使用「FIRST」關鍵字，把新增的欄位放在第一個，或是使用「AFTER」關鍵字，指定新增的欄位要放在哪一個欄位後面。  

In [19]:
# 顯示新增欄位前所有欄位資訊
print("新增欄位前:")
cursor.execute("SHOW FULL COLUMNS FROM birthday")
showdata(cursor)

# 新增 id 欄位並且放在第一欄
cursor.execute("ALTER TABLE birthday ADD id VARCHAR(5) NOT NULL FIRST")
# 新增 job 欄位並且放在 birthdate欄位後面
cursor.execute("ALTER TABLE birthday ADD job VARCHAR(30) NULL AFTER birthdate")
# 新增 age 欄位並且放在 job欄位後面
cursor.execute("ALTER TABLE birthday ADD age TINYINT(3) NULL AFTER job")

# 顯示新增欄位後所有欄位資訊
print("新增欄位後:")
cursor.execute("SHOW FULL COLUMNS FROM birthday")
showdata(cursor)

新增欄位前:


,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,name,varchar(20),utf8_general_ci,NO,,None,,"select,insert,update,references",
1,birthdate,date,None,YES,,None,,"select,insert,update,references",
2,registertime,timestamp,None,NO,,0000-00-00 00:00:00,,"select,insert,update,references",
3,updatetime,timestamp,None,NO,,0000-00-00 00:00:00,on update CURRENT_TIMESTAMP,"select,insert,update,references",


新增欄位後:


,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,id,varchar(5),utf8_general_ci,NO,,None,,"select,insert,update,references",
1,name,varchar(20),utf8_general_ci,NO,,None,,"select,insert,update,references",
2,birthdate,date,None,YES,,None,,"select,insert,update,references",
3,job,varchar(30),utf8_general_ci,YES,,None,,"select,insert,update,references",
4,age,tinyint(3),None,YES,,None,,"select,insert,update,references",
5,registertime,timestamp,None,NO,,0000-00-00 00:00:00,,"select,insert,update,references",
6,updatetime,timestamp,None,NO,,0000-00-00 00:00:00,on update CURRENT_TIMESTAMP,"select,insert,update,references",


### 修改欄位(CHANGE, MODIFY)
如果需要修改欄位的名稱、型態、大小範圍或其它欄位屬性，使用者可以使用下列兩種修改定義來執行修改的工作。「CHANGE」可以修改欄位的名稱與定義，「MODIFY」只能修改欄位的定義，不能修改欄位名稱。

In [20]:
# 顯示修改欄位前所有欄位資訊
print("修改欄位前:")
cursor.execute("SHOW FULL COLUMNS FROM birthday")
showdata(cursor)

# 修改 job 欄位名稱與資料型態
cursor.execute("ALTER TABLE birthday CHANGE job job_rename char(30)")
# 修改 age欄位的資料型態並且放在 birthdate欄位後面
cursor.execute("ALTER TABLE birthday MODIFY age INT(3) AFTER birthdate")

# 顯示修改欄位後所有欄位資訊
print("修改欄位後:")
cursor.execute("SHOW FULL COLUMNS FROM birthday")
showdata(cursor)

修改欄位前:


,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,id,varchar(5),utf8_general_ci,NO,,None,,"select,insert,update,references",
1,name,varchar(20),utf8_general_ci,NO,,None,,"select,insert,update,references",
2,birthdate,date,None,YES,,None,,"select,insert,update,references",
3,job,varchar(30),utf8_general_ci,YES,,None,,"select,insert,update,references",
4,age,tinyint(3),None,YES,,None,,"select,insert,update,references",
5,registertime,timestamp,None,NO,,0000-00-00 00:00:00,,"select,insert,update,references",
6,updatetime,timestamp,None,NO,,0000-00-00 00:00:00,on update CURRENT_TIMESTAMP,"select,insert,update,references",


修改欄位後:


,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,id,varchar(5),utf8_general_ci,NO,,None,,"select,insert,update,references",
1,name,varchar(20),utf8_general_ci,NO,,None,,"select,insert,update,references",
2,birthdate,date,None,YES,,None,,"select,insert,update,references",
3,age,int(3),None,YES,,None,,"select,insert,update,references",
4,job_rename,char(30),utf8_general_ci,YES,,None,,"select,insert,update,references",
5,registertime,timestamp,None,NO,,0000-00-00 00:00:00,,"select,insert,update,references",
6,updatetime,timestamp,None,NO,,0000-00-00 00:00:00,on update CURRENT_TIMESTAMP,"select,insert,update,references",


### 刪除欄位(DROP)
如果要刪除一個表格中不需要的欄位，可以使用**「DROP」**。

In [21]:
# 顯示刪除欄位前所有欄位資訊
print("刪除欄位前:")
cursor.execute("SHOW FULL COLUMNS FROM birthday")
showdata(cursor)

# 刪除 id, job_rename, age 欄位
cursor.execute("ALTER TABLE birthday DROP id")
cursor.execute("ALTER TABLE birthday DROP job_rename")
cursor.execute("ALTER TABLE birthday DROP age")

# 顯示刪除欄位後所有欄位資訊
print("刪除欄位後:")
cursor.execute("SHOW FULL COLUMNS FROM birthday")
showdata(cursor)

刪除欄位前:


,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,id,varchar(5),utf8_general_ci,NO,,None,,"select,insert,update,references",
1,name,varchar(20),utf8_general_ci,NO,,None,,"select,insert,update,references",
2,birthdate,date,None,YES,,None,,"select,insert,update,references",
3,age,int(3),None,YES,,None,,"select,insert,update,references",
4,job_rename,char(30),utf8_general_ci,YES,,None,,"select,insert,update,references",
5,registertime,timestamp,None,NO,,0000-00-00 00:00:00,,"select,insert,update,references",
6,updatetime,timestamp,None,NO,,0000-00-00 00:00:00,on update CURRENT_TIMESTAMP,"select,insert,update,references",


刪除欄位後:


,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,name,varchar(20),utf8_general_ci,NO,,None,,"select,insert,update,references",
1,birthdate,date,None,YES,,None,,"select,insert,update,references",
2,registertime,timestamp,None,NO,,0000-00-00 00:00:00,,"select,insert,update,references",
3,updatetime,timestamp,None,NO,,0000-00-00 00:00:00,on update CURRENT_TIMESTAMP,"select,insert,update,references",


# 索引
「索引」可以預防使用者的資料出現問題，尤其是表格儲存非常大量的紀錄時，建立適當的索引，可以增加查詢與維護資料的效率。索引分為**主索引鍵(primary key)**、**唯一索引(unique index)**與**非唯一索引(non-unique index)**三種。

   + **主索引鍵**：一個表格**只能有一個**，且設定為主索引鍵的欄位值**不可以重複或儲存「NULL」值**，適合使用在類似**編碼**、**代號**這類欄位。  
   + **唯一索引**：一個表格中，設定為唯一索引的欄位值**不可以重複**，但是**可以儲存「NULL」值**，適合用在**電子郵件帳號**的欄位。  
   + **非唯一索引**：設定為非唯一索引的欄位值**可以重複**，也**可以儲存「NULL」值**。

## 建立索引
MySQL提供許多不同的方式讓使用者建立需要的索引。通常在規劃一個資料庫的時候，會把表格所需要的索引一併規劃好，在這樣的情況下，使用者可以把建立索引的定義，加在「CREATE TABLE」敘述中，建立表格的時候就一起把索引建立好；不過也有可能在使用表格一陣子以後，才發覺有建立索引的需求，在這樣的情況下，使用者可以**組合使用「ALTER TABLE」及「CREATE INDEX」**建立需要的索引。

### 使用CREAT TABLE建立索引
建立表格的敘述中，使用者會定義出許多表格所需要的欄位，在欄位的定義中，除了名稱、型態與屬性，還可以加入**索引的定義**。

In [22]:
# 建立表格，並同時建立主索引鍵、唯一索引與非唯一索引
cursor.execute("CREATE TABLE IF NOT EXISTS addressbook( \
                id            INT(5), \
                name          VARCHAR(20) NOT NULL, \
                age           TINYINT(3) UNSIGNED NOT NULL, \
                birthdate     DATE NOT NULL, \
                tel           VARCHAR(10), \
                email         VARCHAR(80), \
                PRIMARY KEY (id), \
                UNIQUE INDEX email_index (email), \
                INDEX name_tel_index (name, tel))")

# 顯示指定表格欄位資訊
cursor.execute("SHOW INDEX FROM addressbook")
showdata(cursor)

# 刪除表格
cursor.execute("DROP TABLE IF EXISTS addressbook")

,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment
0,addressbook,0,PRIMARY,1,id,A,0,None,None,,BTREE,,
1,addressbook,0,email_index,1,email,A,0,None,None,YES,BTREE,,
2,addressbook,1,name_tel_index,1,name,A,0,None,None,,BTREE,,
3,addressbook,1,name_tel_index,2,tel,A,0,None,None,YES,BTREE,,


### 使用ALTER TABLE + CREATE INDEX建立索引
使用「ALTER TABLE」敘述可以建立所有索引，不過**一個「ALTER TABLE」敘述只能建立一個索引**，而**一個「CREATE INDEX」敘述可以建立多個索引，不過只限唯一索引與一般索引**，所以兩者需要互相配合，使用**「ALTER TABLE」敘述建立主索引鍵**，接著再以**「CREATE INDEX」敘述建立其他索引**。

In [23]:
# 建立表格
cursor.execute("CREATE TABLE IF NOT EXISTS addressbook( \
                id            INT(5), \
                name          VARCHAR(20) NOT NULL, \
                age           TINYINT(3) UNSIGNED NOT NULL, \
                birthdate     DATE NOT NULL, \
                tel           VARCHAR(10), \
                email         VARCHAR(80))")

# 顯示建立索引前表格欄位資訊
print("建立索引前:")
cursor.execute("SHOW INDEX FROM addressbook")
showdata(cursor)

# 使用 ALTER TABLE 建立主索引鍵， CREATE INDEX 建立唯一索引與非唯一索引
cursor.execute("ALTER TABLE addressbook ADD PRIMARY KEY (id)")
cursor.execute("CREATE UNIQUE INDEX email_index ON addressbook (email)")
cursor.execute("CREATE INDEX name_tel_index ON addressbook (name, tel)")

# 顯示建立索引後表格欄位資訊
print("建立索引後:")
cursor.execute("SHOW INDEX FROM addressbook")
showdata(cursor)

建立索引前:


""


建立索引後:


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment
0,addressbook,0,PRIMARY,1,id,A,0,None,None,,BTREE,,
1,addressbook,0,email_index,1,email,A,0,None,None,YES,BTREE,,
2,addressbook,1,name_tel_index,1,name,A,0,None,None,,BTREE,,
3,addressbook,1,name_tel_index,2,tel,A,0,None,None,YES,BTREE,,


## 刪除索引
如果一個已經建立好的索引已經不需要了，為了節省儲存的空間，使用者可以使用**「ALTER TABLE」**敘述刪除不需要的索引。

In [24]:
# 顯示刪除索引前表格欄位資訊
print("刪除索引前:")
cursor.execute("SHOW INDEX FROM addressbook")
showdata(cursor)

# 刪除所有索引
cursor.execute("ALTER TABLE addressbook \
                DROP PRIMARY KEY, \
                DROP INDEX email_index, \
                DROP INDEX name_tel_index")

# 顯示刪除索引後表格欄位資訊
print("刪除索引後:")
cursor.execute("SHOW INDEX FROM addressbook")
showdata(cursor)

刪除索引前:


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment
0,addressbook,0,PRIMARY,1,id,A,0,None,None,,BTREE,,
1,addressbook,0,email_index,1,email,A,0,None,None,YES,BTREE,,
2,addressbook,1,name_tel_index,1,name,A,0,None,None,,BTREE,,
3,addressbook,1,name_tel_index,2,tel,A,0,None,None,YES,BTREE,,


刪除索引後:


""


# 查詢資料
在執行資料庫的操作中，查詢算是最常見也是最複雜的工作，所以一個查詢敘述所使用到的子句也最多，下列是查詢敘述的基本語法(要特別注意子句使用的順序)：

| INSTRUCTION | COLLOCATION         | PRIORITY |
|-------------|---------------------|----------|
| SELECT      | columns             |     1    |
| FROM        | table               |     2    |
| WHERE       | conditions          |     3    |
| GROUP BY    | grouping settings   |     4    |
| HAVING      | grouping conditions |     5    |
| ORDER BY    | sorting conditions  |     6    |
| LIMIT       | limiting conditions |     7    |

## 指定資料庫(USE)
一個資料庫伺服器可以建立許多需要的資料庫，所以在使用者執行任何資料庫的操作前，通常要先使用**「USE」**指定使用的資料庫。

In [25]:
# 切換目前使用資料庫
cursor.execute("USE world")

# 檢查目前使用的資料庫
print("目前使用的資料庫:")
cursor.execute("SELECT database()")
showdata(cursor)

目前使用的資料庫:


,database()
0,world


## 顯示內容(SELECT)
一個SQL查詢敘述一定要以**「SELECT」子句開始**，再搭配其它的子句完成查詢資料的工作。使用者可以單獨使用「SELECT」子句，只不過這樣的用法跟資料庫一點關係都沒有，它只不過把使用者輸入的內容顯示出來而已。

In [26]:
# 單純印出
cursor.execute("SELECT 'My name is Simon Johnson', 35 * 12")
showdata(cursor)

,My name is Simon Johnson,35 * 12
0,My name is Simon Johnson,420


## 基本查詢(FROM)
一般所謂的查詢敘述，通常是查詢資料庫中的資料，所以**「SELECT」子句會搭配「FROM」子句**來使用，而「SELECT」後面可以指定**「\*」**表示要查詢指定表格的**所有欄位**或是單純**指定想要的欄位**。  

若想取得欄位中部重複的資料則可以使用**「DISTINCT」**的敘述語法。  

如果想選擇的表格不在目前使用的資料庫底下，則可以使用**「database.table」**的敘述語法取得表格內容。  

除了查詢表格中的欄位外，使用者可以加入任何需要的**數學運算子**，此外也能為「SELECT」後面查詢的資料取一個自己想要的名稱。

**數學運算子**  

| OPERATOR | MEANING          | PRIORITY |
|----------|------------------|----------|
| %, MOD   | Modulo           |     1    |
| *        | Multiplication   |     1    |
| /        | Division         |     1    |
| DIV      | Integer Division |     1    |
| +        | Addition         |     2    |
| -        | Subtration       |     2    |

In [27]:
# 印出 city 表格的所有欄位(city 表格在 world 資料庫底下)
print("world資料庫內city表格的所有欄位:")
cursor.execute("SELECT * FROM city")
showdata(cursor, 5)

# 印出 city 表格的指定欄位並設定為不重複
print("world資料庫內city表格的指定欄位並設定為不重複:")
cursor.execute("SELECT DISTINCT CountryCode FROM city")
showdata(cursor, 5)

# 印出 cmdev 資料庫底下的 emp 表格的所有欄位
print("cmdev資料庫內emp表格的所有欄位:")
cursor.execute("SELECT * FROM cmdev.emp")
showdata(cursor, 5)

# 印出 cmdev 資料庫底下的 emp 表格的指定欄位並取別名
print("cmdev資料庫內emp表格的指定欄位:")
cursor.execute("SELECT ename, salary, salary*12 + salary / 2  AS annual_salary FROM cmdev.emp")
showdata(cursor, 5)

world資料庫內city表格的所有欄位:


,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200


world資料庫內city表格的指定欄位並設定為不重複:


,CountryCode
0,AFG
1,NLD
2,ANT
3,ALB
4,DZA


cmdev資料庫內emp表格的所有欄位:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20.0
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20.0
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0


cmdev資料庫內emp表格的指定欄位:


,ename,salary,annual_salary
0,SMITH,800.0,10000.0
1,ALLEN,1600.0,20000.0
2,WARD,1250.0,15625.0
3,JONES,2975.0,37187.5
4,MARTIN,1250.0,15625.0


## 條件查詢(WHERE)
使用「SELECT」和「FROM」執行的查詢敘述，是把在「FROM」子句指定表格裡所有的紀錄傳回來。資料庫最大的好處就是可以隨時依照需要查詢部份紀錄資料，而使用者可以使用**「WHERE」**子句搭配**比較運算子**、**邏輯運算子**或是**其它條件運算子**執行查詢條件的設定。 

比較特別的是**「NULL」值的判斷**，不可以使用判斷一般資料的條件設定，必須使用**「<=>、IS」或「IS NOT」**。  

在使用字串資料的條件判斷時，會有一種很常見、也比較特殊的需求，像是「想要查詢名稱以某字元開始的字串」就需要使用到**「Like」與兩種樣版字元「%, \_ 」**，而**「%」代表零到多個任何字元**，**「\_ 」代表一個任何字元**。

**比較運算子**  

| OPERATOR | MEANING                  | PRIORITY |
|----------|--------------------------|----------|
| =        | Equal to                 |     1    |
| <=>      | Equal to(work on NULL)   |     1    |
| !=       | Unequal to               |     1    |
| <        | Less than                |     1    |
| <=       | Less than or Equal to    |     1    |
| >        | Greater to               |     1    |
| >=       | Greater than or Equal to |     1    |


**邏輯運算子**  

| OPERATOR | MEANING      | PRIORITY |
|----------|--------------|----------|
| NOT      | Not          |     1    |
| &&, AND  | And          |     2    |
| XOR      | Exclusive Or |     3    |
| \|\|, OR | Or           |     4    |


**其它條件運算子**  

| OPERATOR         | PRIORITY |
|------------------|----------|
| IN               |     1    |
| IS(NOT)          |     1    |
| LIKE             |     1    |
| BETWEEN...AND... |     2    |

In [28]:
# 使用運算子篩出想要的資料
print("人口數界於10000到30000，國家代碼不為('FRO', 'MCO', 'PYF')且區域與名稱內沒有任何?的資料:")
cursor.execute("SELECT * \
                FROM city \
                WHERE (Population BETWEEN 10000 AND 30000) AND (NOT CountryCode IN ('FRO', 'MCO', 'PYF')) \
                                                           AND (NOT District Like '%?%') \
                                                           AND (NOT Name Like '%?%')")
showdata(cursor)


# 使用配合 NULL 的運算子
print("平均壽命為NULL的資料:")
cursor.execute("SELECT Code, Continent, LifeExpectancy \
                FROM country \
                WHERE LifeExpectancy IS NULL")
showdata(cursor)

人口數界於10000到30000，國家代碼不為('FRO', 'MCO', 'PYF')且區域與名稱內沒有任何?的資料:


,ID,Name,CountryCode,District,Population
0,55,Andorra la Vella,AND,Andorra la Vella,21189
1,192,Thimphu,BTN,Thimphu,22000
2,535,Saint Helier,GBR,Jersey,27523
3,538,Bandar Seri Begawan,BRN,Brunei and Muara,21484
4,553,George Town,CYM,Grand Cayman,19600
5,583,Avarua,COK,Rarotonga,11900
6,586,Roseau,DMA,St George,16243
7,917,Nuuk,GRL,Kitaa,13445
8,919,Basse-Terre,GLP,Basse-Terre,12433
9,2429,Kuwait,KWT,al-Asima,28859


平均壽命為NULL的資料:


,Code,Continent,LifeExpectancy
0,ATA,Antarctica,None
1,ATF,Antarctica,None
2,BVT,Antarctica,None
3,CCK,Oceania,None
4,CXR,Oceania,None
5,FLK,South America,None
6,HMD,Antarctica,None
7,IOT,Africa,None
8,NFK,Oceania,None
9,NIU,Oceania,None


## 分組查詢(GROUP BY, HAVING)
如果希望依照指定的資料來計算分組統計與分析資訊可以使用**「GROUP BY」子句指定分組的設定**，而群組函式的條件設定則要使用**「HAVING」子句**中。

In [29]:
# 依大洲及地區分群遞減輸出總人口數(且帶有小計)
print("依大洲及地區分群遞減輸出總人口數:")
cursor.execute("SELECT Continent, Region, SUM(Population) Popsum \
                FROM country \
                GROUP BY Continent, Region DESC WITH ROLLUP \
                HAVING SUM(Population) > 200000000")
showdata(cursor)

依大洲及地區分群遞減輸出總人口數:


,Continent,Region,Popsum
0,Asia,Southern and Central Asia,1490776000
1,Asia,Southeast Asia,518541000
2,Asia,Eastern Asia,1507328000
3,Asia,None,3705025700
4,Europe,Eastern Europe,307026000
5,Europe,None,730074600
6,North America,North America,309632000
7,North America,None,482993000
8,Africa,Western Africa,221672000
9,Africa,Eastern Africa,246999000


## 排序查詢(ORDER BY)
在執行任何一個查詢以後，MySQL傳回的資料是依照資料新增到表格中的順序，而當使用者查詢時通常會有**資料排序**上的需求，因此可以使用**「ORDER BY」**子句來達成，通常與會使用**「ASC」(遞增)或「DESC」(遞減)**兩種屬性配合使用。

In [30]:
# 依使用者排序輸出(先依國家代碼遞增，若有相同，再依人口數遞減)
print("依國家代碼遞增，人口數遞減排序輸出資料:")
cursor.execute("SELECT * \
                FROM city \
                ORDER BY CountryCode ASC, Population DESC")
showdata(cursor)

依國家代碼遞增，人口數遞減排序輸出資料:


,ID,Name,CountryCode,District,Population
0,129,Oranjestad,ABW,?–,29034
1,1,Kabul,AFG,Kabol,1780000
2,2,Qandahar,AFG,Qandahar,237500
3,3,Herat,AFG,Herat,186800
4,4,Mazar-e-Sharif,AFG,Balkh,127800
5,56,Luanda,AGO,Luanda,2022000
6,57,Huambo,AGO,Huambo,163100
7,58,Lobito,AGO,Benguela,130000
8,59,Benguela,AGO,Benguela,128300
9,60,Namibe,AGO,Namibe,118200


## 限制查詢(LIMIT)
在執行一個查詢敘述後，資料庫會將查詢的資料傳回來給；若有**限制數量**的需求(如前三名)可以使用**「LIMIT」**子句指定回傳紀錄的數量。

In [31]:
# 依使用者設定限制查詢
print("限制輸出第6-10筆的資料:")
cursor.execute("SELECT * \
                FROM city \
                LIMIT 5, 5")
showdata(cursor)

# 依使用者設定限制查詢
print("限制輸出人口數最少的5筆資料:")
cursor.execute("SELECT * \
                FROM city \
                ORDER BY Population ASC \
                LIMIT 5")
showdata(cursor)

限制輸出第6-10筆的資料:


,ID,Name,CountryCode,District,Population
0,6,Rotterdam,NLD,Zuid-Holland,593321
1,7,Haag,NLD,Zuid-Holland,440900
2,8,Utrecht,NLD,Utrecht,234323
3,9,Eindhoven,NLD,Noord-Brabant,201843
4,10,Tilburg,NLD,Noord-Brabant,193238


限制輸出人口數最少的5筆資料:


,ID,Name,CountryCode,District,Population
0,2912,Adamstown,PCN,?–,42
1,2317,West Island,CCK,West Island,167
2,3333,Fakaofo,TKL,Fakaofo,300
3,3538,Citt?? del Vaticano,VAT,?–,455
4,2316,Bantam,CCK,Home Island,503


## 結合查詢 (INNER JOIN, OUTER JOIN)
要查詢的資料，來自於一個以上的表格，而且兩個表格之間具有「對照」情形。

### 內部結合 (INNER JOIN)
使用**「INNER JOIN」**的查詢，一定要符合**結合條件**的資料才會出現，換句話說就是A表格與B表格都有同樣的值時才會被回傳。

In [32]:
# 查詢 country 表格內指定欄位
print("country 表格:")
cursor.execute("SELECT Code, Name \
                FROM country")
showdata(cursor, header=5)

# 查詢 city 表格內指定欄位
print("city 表格:")
cursor.execute("SELECT CountryCode, Population \
                FROM city")
showdata(cursor, header=5)

country 表格:


,Code,Name
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALB,Albania


city 表格:


,CountryCode,Population
0,AFG,1780000
1,AFG,237500
2,AFG,186800
3,AFG,127800
4,NLD,731200


In [33]:
# 結合查詢
print("結合查詢:")
cursor.execute("SELECT country.Code, country.Name, city.Population \
                FROM country, city \
                WHERE country.Code = city.CountryCode")
showdata(cursor, header=5)

# 利用 INNER JOIN 做結合查詢
print("INNER JOIN查詢:")
cursor.execute("SELECT country.Code, country.Name, city.Population \
                FROM country INNER JOIN city ON country.Code = city.CountryCode")
showdata(cursor, header=5)

結合查詢:


,Code,Name,Population
0,AFG,Afghanistan,1780000
1,AFG,Afghanistan,237500
2,AFG,Afghanistan,186800
3,AFG,Afghanistan,127800
4,NLD,Netherlands,731200


INNER JOIN查詢:


,Code,Name,Population
0,AFG,Afghanistan,1780000
1,AFG,Afghanistan,237500
2,AFG,Afghanistan,186800
3,AFG,Afghanistan,127800
4,NLD,Netherlands,731200


### 外部結合 (OUTER JOIN)
在結合查詢的應用中，如果希望**不符合結合條件**的資料也要出現的話，就要換成使用**「OUTER JOIN」**來執行結合查詢。而外部結合又分為以前表格為主的**「LEFT OUTER JOIN」**和以後表格為主的**「RIGHT OUTER JOIN」**。

In [34]:
# 切換目前使用資料庫
cursor.execute("USE cmdev")

# 查詢 emp 表格內指定欄位
print("emp 表格:")
cursor.execute("SELECT empno, ename, deptno \
                FROM emp")
showdata(cursor)

# 查詢 dept 表格內指定欄位
print("dept 表格:")
cursor.execute("SELECT deptno, dname \
                FROM dept")
showdata(cursor)

emp 表格:


,empno,ename,deptno
0,7369,SMITH,20.0
1,7499,ALLEN,30.0
2,7521,WARD,30.0
3,7566,JONES,20.0
4,7654,MARTIN,30.0
5,7698,BLAKE,NaN
6,7782,CLARK,10.0
7,7788,SCOTT,20.0
8,7839,KING,10.0
9,7844,TURNER,30.0


dept 表格:


,deptno,dname
0,10,ACCOUNTING
1,20,RESEARCH
2,30,SALES
3,40,OPERATIONS
4,50,IT


In [35]:
# 利用 INNER JOIN 做結合查詢
print("INNER JOIN查詢，缺少兩筆未吻合資料:")
cursor.execute("SELECT emp.empno, emp.ename, dept.deptno, dept.dname\
                FROM emp INNER JOIN dept ON emp.deptno = dept.deptno")
showdata(cursor)

# 利用 LEFT OUTER JOIN 做結合查詢
print("LEFT OUTER JOIN查詢，未缺少資料，以前表格為主:")
cursor.execute("SELECT emp.empno, emp.ename, dept.deptno, dept.dname\
                FROM emp LEFT OUTER JOIN dept ON emp.deptno = dept.deptno")
showdata(cursor)

# 利用 RIGHT OUTER JOIN 做結合查詢
print("RIGHT OUTER JOIN查詢，未缺少資料，以後表格為主:")
cursor.execute("SELECT emp.empno, emp.ename, dept.deptno, dept.dname\
                FROM emp RIGHT OUTER JOIN dept ON emp.deptno = dept.deptno")
showdata(cursor)

INNER JOIN查詢，缺少兩筆未吻合資料:


,empno,ename,deptno,dname
0,7369,SMITH,20,RESEARCH
1,7499,ALLEN,30,SALES
2,7521,WARD,30,SALES
3,7566,JONES,20,RESEARCH
4,7654,MARTIN,30,SALES
5,7782,CLARK,10,ACCOUNTING
6,7788,SCOTT,20,RESEARCH
7,7839,KING,10,ACCOUNTING
8,7844,TURNER,30,SALES
9,7876,ADAMS,20,RESEARCH


LEFT OUTER JOIN查詢，未缺少資料，以前表格為主:


,empno,ename,deptno,dname
0,7782,CLARK,10.0,ACCOUNTING
1,7839,KING,10.0,ACCOUNTING
2,7934,MILLER,10.0,ACCOUNTING
3,7369,SMITH,20.0,RESEARCH
4,7566,JONES,20.0,RESEARCH
5,7788,SCOTT,20.0,RESEARCH
6,7876,ADAMS,20.0,RESEARCH
7,7902,FORD,20.0,RESEARCH
8,7499,ALLEN,30.0,SALES
9,7521,WARD,30.0,SALES


RIGHT OUTER JOIN查詢，未缺少資料，以後表格為主:


,empno,ename,deptno,dname
0,7369.0,SMITH,20,RESEARCH
1,7499.0,ALLEN,30,SALES
2,7521.0,WARD,30,SALES
3,7566.0,JONES,20,RESEARCH
4,7654.0,MARTIN,30,SALES
5,7782.0,CLARK,10,ACCOUNTING
6,7788.0,SCOTT,20,RESEARCH
7,7839.0,KING,10,ACCOUNTING
8,7844.0,TURNER,30,SALES
9,7876.0,ADAMS,20,RESEARCH


## 合併查詢 (UNION)
在關聯式資料庫中，因為表格的設計，常會使用結合查詢來取得需要的資料，結合查詢指的是在**「一個」查詢敘述中使用「多個」資料表**。而現在要討論的**「UNION」**查詢，指的是把**一個以上的查詢敘述所得到的結果合併為一個**。

In [36]:
# 切換目前使用資料庫
cursor.execute("USE world")

# 多條件查詢
print("使用WHERE進行多條件查詢:")
cursor.execute("SELECT Region, Name, Population \
                FROM country \
                WHERE (Region = 'Southeast Asia' AND Population < 2000000) OR \
                      (Region = 'Eastern Asia' AND Population < 1000000)")
showdata(cursor, header=5)

# UNION 查詢
print("使用UNION進行多條件查詢:")
cursor.execute("SELECT Region, Name, Population \
                FROM country \
                WHERE Region = 'Southeast Asia' AND Population < 2000000 \
                UNION \
                SELECT Region, Name, Population \
                FROM country \
                WHERE Region = 'Eastern Asia' AND Population < 1000000")
showdata(cursor, header=5)

使用WHERE進行多條件查詢:


,Region,Name,Population
0,Southeast Asia,Brunei,328000
1,Eastern Asia,Macao,473000
2,Southeast Asia,East Timor,885000


使用UNION進行多條件查詢:


,Region,Name,Population
0,Southeast Asia,Brunei,328000
1,Southeast Asia,East Timor,885000
2,Eastern Asia,Macao,473000


## 子查詢

### 子查詢與「WHERE」&「HAVING」
子查詢大部份使用在提供判斷條件用的資料，所以在**「WHERE」**和**「HAVING」**子句中，都可能出現子查詢，在「WHERE」和「HAVING」子句中，可以使用許多不同的運算子來設定判斷的條件。

譬如使用**比較運算子**的時候，要提供一個資料讓運算子判斷條件是否符合並且同時要特別注意子查詢回傳的資料是否符合規定(只能回傳一個值)。比較運算子與子查詢搭配使用的時候，另外還提供**「ALL」**、**「ANY」**與**「SOME」**三個運算子。其中「ANY」與「SOME」運算子的效果是一樣的。

除了一般的比較運算子外，也經常使用**「IN」運算子**案執行多個資料的比較。也可以使用子查詢提供「IN」、「NOT IN」運算子的判斷資料。

In [37]:
# 子查詢搭配 WHERE
print("子查詢搭配WHERE:")
cursor.execute("SELECT Name, GNP \
                FROM country \
                WHERE GNP = (SELECT MAX(GNP) \
                             FROM country)")
showdata(cursor)

# 子查詢搭配 ALL
print("子查詢搭配ALL:")
cursor.execute("SELECT Code, Population \
                FROM country \
                WHERE Population > ALL (SELECT Population \
                                        FROM city)")
showdata(cursor, header=5)

# 子查詢搭配 ANY
print("子查詢搭配ANY:")
cursor.execute("SELECT Code, Population \
                FROM country \
                WHERE Population < ANY (SELECT Population \
                                        FROM city)")
showdata(cursor, header=5)

# 子查詢搭配 IN
print("子查詢搭配IN:")
cursor.execute("SELECT Name \
                FROM country \
                WHERE code NOT IN (SELECT CountryCode \
                                   FROM city \
                                   WHERE Population > 10000)")
showdata(cursor, header=5)

子查詢搭配WHERE:


,Name,GNP
0,United States,8510700.0


子查詢搭配ALL:


,Code,Population
0,AFG,22720000
1,AGO,12878000
2,ARG,37032000
3,AUS,18886000
4,BFA,11937000


子查詢搭配ANY:


,Code,Population
0,ABW,103000
1,AIA,8000
2,ALB,3401200
3,AND,78000
4,ANT,217000


子查詢搭配IN:


,Name
0,Anguilla
1,Netherlands Antilles
2,American Samoa
3,Antarctica
4,French Southern territories


In [38]:
# 多欄位子查詢
print("多欄位子查詢:")
cursor.execute("SELECT Name \
                FROM country \
                WHERE (Region, GovernmentForm) = (SELECT Region, GovernmentForm \
                                                  FROM country \
                                                  WHERE name = 'Taiwan')")
showdata(cursor)


# 多欄位子查詢搭配 IN
print("多欄位子查詢搭配IN:")
cursor.execute("SELECT Continent, Name, GNP \
                FROM country \
                WHERE (Continent, GNP) IN (SELECT Continent, MAX(GNP) \
                                           FROM country \
                                           GROUP BY Continent) \
                ORDER BY GNP DESC")
showdata(cursor)

多欄位子查詢:


,Name
0,South Korea
1,Mongolia
2,Taiwan


多欄位子查詢搭配IN:


,Continent,Name,GNP
0,North America,United States,8510700.0
1,Asia,Japan,3787042.0
2,Europe,Germany,2133367.0
3,South America,Brazil,776739.0
4,Oceania,Australia,351182.0
5,Africa,South Africa,116729.0
6,Antarctica,Antarctica,0.0
7,Antarctica,French Southern territories,0.0
8,Antarctica,Bouvet Island,0.0
9,Antarctica,Heard Island and McDonald Islands,0.0


### 子查詢與「SELECT」


In [39]:
# 子查詢搭配 SELECT
print("子查詢搭配SELECT:")
cursor.execute("SELECT CONCAT('The GNP of Japan is ', (SELECT GNP \
                                                       FROM country \
                                                       WHERE Name = 'Japan')) String")
showdata(cursor)

子查詢搭配SELECT:


,String
0,The GNP of Japan is 3787042.00


### 子查詢與「FROM」
子查詢可以使用在「WHERE」與「HAVGIN」子句中用來設定條件，還有使用在「SELECT」子句中用來傳回需要的資料。除了這兩種用法外，子查
詢還可以使用在「FROM」子句。通常會在查詢敘述的「FROM」子句中，指定需要的表格名稱，在「FROM」子句中使用子查詢，這個子查詢回傳的結果會被當成一個「表格」，而通常不常使用這種用法。

In [40]:
# 子查詢搭配 FROM
print("子查詢搭配FROM:")
cursor.execute("SELECT Name, GNP \
                FROM (SELECT * \
                      FROM country \
                      WHERE Continent = 'Asia') asiacountry \
                ORDER BY GNP DESC")
showdata(cursor, header=5)

子查詢搭配FROM:


,Name,GNP
0,Japan,3787042.0
1,China,982268.0
2,India,447114.0
3,South Korea,320749.0
4,Taiwan,256254.0


## 資料維護

In [41]:
# 刪除表格前資料庫內所有表格
print("刪除表格前:")
cursor.execute("USE world")
cursor.execute("SHOW TABLES")
showdata(cursor, header='all')
cursor.execute("USE cmdev")
cursor.execute("SHOW TABLES")
showdata(cursor, header='all')

# 刪除所有表格
cursor.execute("DROP TABLE IF EXISTS world.country_copy")
cursor.execute("DROP TABLE IF EXISTS cmdev.dept_copy")
cursor.execute("DROP TABLE IF EXISTS cmdev.emp_copy")
cursor.execute("DROP TABLE IF EXISTS cmdev.travel_copy")

# 刪除表格後資料庫內所有表格
print("刪除表格後:")
cursor.execute("USE world")
cursor.execute("SHOW TABLES")
showdata(cursor, header='all')
cursor.execute("USE cmdev")
cursor.execute("SHOW TABLES")
showdata(cursor, header='all')

刪除表格前:


,Tables_in_world
0,city
1,country
2,country_copy
3,countrylanguage
4,scaleview


,Tables_in_cmdev
0,binarytable
1,bittable
2,debug
3,dept
4,dept_copy
5,deptlog
6,deptlog_copy
7,dttable
8,emp
9,emp_copy


刪除表格後:


,Tables_in_world
0,city
1,country
2,countrylanguage
3,scaleview


,Tables_in_cmdev
0,binarytable
1,bittable
2,debug
3,dept
4,deptlog
5,deptlog_copy
6,dttable
7,emp
8,emp_copy2
9,empdept20view


In [42]:
# 建立一些複製表格以免破壞原有資料集

# 使用查詢表格來建立 world.country 的複製品
cursor.execute("CREATE TABLE IF NOT EXISTS world.country_copy( \
                PRIMARY KEY (code)) \
                SELECT * \
                FROM world.country")
cursor.execute("TRUNCATE TABLE world.country_copy")

# 使用查詢表格來建立 cmdev.dept 的複製品
cursor.execute("CREATE TABLE IF NOT EXISTS cmdev.dept_copy( \
                PRIMARY KEY (deptno)) \
                SELECT * \
                FROM cmdev.dept")

# 使用查詢表格來建立 cmdev.emp 的複製品
cursor.execute("CREATE TABLE IF NOT EXISTS cmdev.emp_copy( \
                PRIMARY KEY (empno)) \
                SELECT * \
                FROM cmdev.emp")

# 使用查詢表格來建立 cmdev.travel 的複製品
cursor.execute("CREATE TABLE IF NOT EXISTS cmdev.travel_copy( \
                PRIMARY KEY (empno, location)) \
                SELECT * \
                FROM cmdev.travel")

### 新增資料 (INSERT INTO)
新增資料到資料庫的表格中使用**「INSERT」**敘述，使用這個語法新增紀錄的時候，要特別注意表格的**欄位個數與順序**，如果資料個數與欄位個數不一樣的話，就會發生錯誤。除了明確的指定新增紀錄的每一個欄位資料外，你也可以使用**「DEFAULT」**關鍵字，讓MySQL為你寫入在設計表格的時候，為欄位指定的預設值。

新增敘述的另外一種語法，可以指定新增紀錄的欄位個數和順序，在額外為這個新增敘述指定欄位以後，指定儲存資料的時候就要依照指定的欄位個數與順序，如果沒有依照指定的欄位個數與順序，就會發生錯誤。

In [43]:
cursor.execute("USE cmdev")

# 新增資料前
print("新增資料前:")
cursor.execute("SELECT * FROM dept_copy")
showdata(cursor)

# 不指定欄位新增資料
print("不指定欄位新增資料:")
cursor.execute("INSERT INTO dept_copy VALUES (60, 'EDU', 'HOUSTON')")
cursor.execute("INSERT INTO dept_copy VALUES (70, 'MARKETING', DEFAULT)")
cursor.execute("SELECT * FROM cmdev.dept_copy")
showdata(cursor)

# 指定欄位新增資料
print("指定欄位新增資料:")
cursor.execute("INSERT INTO dept_copy (deptno, dname, location) VALUES (80, 'HR', 'LOS ANGELES')")
cursor.execute("INSERT INTO dept_copy (deptno, dname) VALUES (90, 'RD')")  # 未寫入的欄位直接帶入DEFAULT
cursor.execute("SELECT * FROM cmdev.dept_copy")
showdata(cursor)

# 指定欄位新增資料(使用SET)
print("指定欄位新增資料(使用SET):")
cursor.execute("INSERT INTO dept_copy SET deptno=100, dname='SHIPPING', location='DALLAS'")
cursor.execute("SELECT * FROM dept_copy")
showdata(cursor)

新增資料前:


,deptno,dname,location
0,10,ACCOUNTING,NEW YORK
1,20,RESEARCH,DALLAS
2,30,SALES,CHICAGO
3,40,OPERATIONS,BOSTON
4,50,IT,NEW YORK


不指定欄位新增資料:


,deptno,dname,location
0,10,ACCOUNTING,NEW YORK
1,20,RESEARCH,DALLAS
2,30,SALES,CHICAGO
3,40,OPERATIONS,BOSTON
4,50,IT,NEW YORK
5,60,EDU,HOUSTON
6,70,MARKETING,None


指定欄位新增資料:


,deptno,dname,location
0,10,ACCOUNTING,NEW YORK
1,20,RESEARCH,DALLAS
2,30,SALES,CHICAGO
3,40,OPERATIONS,BOSTON
4,50,IT,NEW YORK
5,60,EDU,HOUSTON
6,70,MARKETING,None
7,80,HR,LOS ANGELES
8,90,RD,None


指定欄位新增資料(使用SET):


,deptno,dname,location
0,10,ACCOUNTING,NEW YORK
1,20,RESEARCH,DALLAS
2,30,SALES,CHICAGO
3,40,OPERATIONS,BOSTON
4,50,IT,NEW YORK
5,60,EDU,HOUSTON
6,70,MARKETING,None
7,80,HR,LOS ANGELES
8,90,RD,None
9,100,SHIPPING,DALLAS


In [44]:
# 新增資料前
print("新增資料前:")
cursor.execute("SELECT * FROM emp_copy")
showdata(cursor)

# 指定欄位新增多筆資料
print("指定欄位新增多筆資料:")
cursor.execute("INSERT INTO emp_copy (empno, ename, job, manager, hiredate, salary, comm, deptno) \
                VALUES (8001, 'SIMON', 'MANAGER', 7369, '2001-02-03', 3300, NULL, 50), \
                       (8002, 'JOHN', 'PROGRAMMER', 8001, '2002-01-01', 2300, NULL, 50), \
                       (8003, 'GREEN', 'ENGINEER', 8001, '2003-05-01', 2000, NULL, 50)")
cursor.execute("SELECT * FROM emp_copy")
showdata(cursor, header='all')

新增資料前:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20.0
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20.0
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,NaN
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10.0
7,7788,SCOTT,ANALYST,7566.0,1987-04-19,3000.0,NaN,20.0
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10.0
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0


指定欄位新增多筆資料:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20.0
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20.0
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,NaN
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10.0
7,7788,SCOTT,ANALYST,7566.0,1987-04-19,3000.0,NaN,20.0
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10.0
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0


#### 新增資料與子查詢
使用 INSERT 敘述執行新增記錄的工作時，通常是直接指定新增記錄的資料。如果你要新增的資料，需要執行一個查詢來取得的話，就可以搭配子查詢來簡化新增紀錄的工作。

In [45]:
cursor.execute("USE world")

# 新增資料前
print("新增資料前:")
cursor.execute("SELECT * FROM country_copy")
showdata(cursor)

# 利用子查詢與 INSERT INTO 新增資料
print("利用子查詢與INSERT INTO新增資料:")
cursor.execute("INSERT INTO country_copy \
               (SELECT * \
                FROM country \
                WHERE Continent = 'Asia')")
cursor.execute("SELECT * FROM country_copy")
showdata(cursor, header=5)

新增資料前:


""


利用子查詢與INSERT INTO新增資料:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1,AF
1,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971.0,2441000,74.1,37966.0,36846.0,Al-Imarat al-??Arabiya al-Muttahida,Emirate Federation,Zayid bin Sultan al-Nahayan,65,AE
2,ARM,Armenia,Asia,Middle East,29800.0,1991.0,3520000,66.4,1813.0,1627.0,Hajastan,Republic,Robert Kot??arjan,126,AM
3,AZE,Azerbaijan,Asia,Middle East,86600.0,1991.0,7734000,62.9,4127.0,4100.0,Az??rbaycan,Federal Republic,Heyd??r ??liyev,144,AZ
4,BGD,Bangladesh,Asia,Southern and Central Asia,143998.0,1971.0,129155000,60.2,32852.0,31966.0,Bangladesh,Republic,Shahabuddin Ahmad,150,BD


### 更新資料 (ON DUPLICATE KEY UPDATE, REPLACE, UPDATE)
使用**「INSERT」**敘述新增紀錄的時候，還可以視需要在最後搭配一串關鍵字**「ON DUPLICATE KEY UPDATE」**，它可以用來指定在**違反重複索引值的規定**時要執行的修改。

除了使用**「INSERT」**敘述新增紀錄外，**「REPLACE」**敘述同樣可以新增紀錄，它們的語法幾乎相同，但**「REPLACE」**敘述**可以違反複索引值的規定**，將指定的資料整個覆蓋過去。

另外也可以使用**「UPDATE」**敘述修改資料，通常會搭配使用**「WHERE」**子句來指定要修改的紀錄，執行修改的時候使用「WHERE」子句是一般最常見的用法，在處理一些比較特殊的修改需求時，也會搭配**「ORDER BY」**與**「LIMIT」**子句。

In [46]:
cursor.execute("USE cmdev")

# 更新資料前
print("更新資料前:")
cursor.execute("DESC travel_copy")
showdata(cursor)
cursor.execute("SELECT * FROM travel_copy")
showdata(cursor)

# 使用 INSERT INTO 更新資料
print("使用INSERT INTO更新資料:")
try:
    cursor.execute("INSERT INTO travel_copy (empno, location, counter) \
                    VALUES (7566, 'BOSTON', 4)")
    cursor.execute("SELECT * FROM travel_copy")
    showdata(cursor)
    
except:
    print("ERROR: 使用INSERT敘述更新時違反主索引規定! 請加上 ON DUPLICATE KEY UPDATE \n")

print("使用INSERT INTO & ON DUPLICATE KEY UPDATE更新資料:")
cursor.execute("INSERT INTO travel_copy (empno, location, counter) \
                VALUES (7566, 'BOSTON', 1), \
                       (7654, 'NEW YORK', 1) \
                ON DUPLICATE KEY UPDATE counter = counter + 1 ")
cursor.execute("SELECT * FROM travel_copy")
showdata(cursor)

# 使用 REPLACE 更新資料
print("使用REPLACE更新資料:")
cursor.execute("REPLACE travel_copy (empno, location, counter) \
                VALUES (7369, 'CHICAGO', 10), \
                       (7777, 'DALLAS', 5)")
cursor.execute("SELECT * FROM travel_copy")
showdata(cursor)

# 使用 UPDATE 更新資料
print("使用UPDATE更新資料:")
cursor.execute("UPDATE travel_copy \
                SET counter = 0 \
                WHERE location = 'DALLAS' \
                ORDER BY counter DESC \
                LIMIT 1")
cursor.execute("SELECT * FROM travel_copy")
showdata(cursor)

更新資料前:


,Field,Type,Null,Key,Default,Extra
0,empno,int(11),NO,PRI,None,
1,location,varchar(16),NO,PRI,None,
2,counter,int(11),NO,,None,


,empno,location,counter
0,7369,CHICAGO,1
1,7499,DALLAS,1
2,7521,DALLAS,2
3,7566,BOSTON,1
4,7654,NEW YORK,1


使用INSERT INTO更新資料:
ERROR: 使用INSERT敘述更新時違反主索引規定! 請加上 ON DUPLICATE KEY UPDATE 

使用INSERT INTO & ON DUPLICATE KEY UPDATE更新資料:


,empno,location,counter
0,7369,CHICAGO,1
1,7499,DALLAS,1
2,7521,DALLAS,2
3,7566,BOSTON,2
4,7654,NEW YORK,2


使用REPLACE更新資料:


,empno,location,counter
0,7369,CHICAGO,10
1,7499,DALLAS,1
2,7521,DALLAS,2
3,7566,BOSTON,2
4,7654,NEW YORK,2
5,7777,DALLAS,5


使用UPDATE更新資料:


,empno,location,counter
0,7369,CHICAGO,10
1,7499,DALLAS,1
2,7521,DALLAS,2
3,7566,BOSTON,2
4,7654,NEW YORK,2
5,7777,DALLAS,0


#### 更新資料與子查詢
使用「UPDATE」敘述執行修改資料時，如果沒有使用「WHERE」子句指定修改的條件，「UPDATE」敘述會**修改表格中所有的紀錄**。所以執行修改紀錄資料的時候，通常會使用「WHERE」子句指定修改的條件。在「UPDATE」敘述的「WHERE」子句，也可以使用子查詢提供判斷條件的資料。

In [47]:
cursor.execute("USE world")

# 新增資料前
print("新增資料前:")
cursor.execute("SELECT * FROM country_copy")
showdata(cursor, header=5)

# 利用子查詢與 REPLACE 更新資料
print("利用子查詢與REPLACE更新資料:")
cursor.execute("REPLACE country_copy \
               (SELECT * \
                FROM country \
                WHERE Continent = 'Africa')")
cursor.execute("SELECT * FROM country_copy")
showdata(cursor, header=5)

# 利用子查詢與 UPDATE 更新資料
print("利用子查詢與UPDATE更新資料:")
cursor.execute("UPDATE country_copy \
                SET GNP = GNP * 2 \
                WHERE Code IN (SELECT CountryCode \
                               FROM city \
                               WHERE Population > 1000000)")
cursor.execute("SELECT * FROM country_copy")
showdata(cursor, header=5)

新增資料前:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1,AF
1,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971.0,2441000,74.1,37966.0,36846.0,Al-Imarat al-??Arabiya al-Muttahida,Emirate Federation,Zayid bin Sultan al-Nahayan,65,AE
2,ARM,Armenia,Asia,Middle East,29800.0,1991.0,3520000,66.4,1813.0,1627.0,Hajastan,Republic,Robert Kot??arjan,126,AM
3,AZE,Azerbaijan,Asia,Middle East,86600.0,1991.0,7734000,62.9,4127.0,4100.0,Az??rbaycan,Federal Republic,Heyd??r ??liyev,144,AZ
4,BGD,Bangladesh,Asia,Southern and Central Asia,143998.0,1971.0,129155000,60.2,32852.0,31966.0,Bangladesh,Republic,Shahabuddin Ahmad,150,BD


利用子查詢與REPLACE更新資料:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
1,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,Jos?? Eduardo dos Santos,56.0,AO
2,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971.0,2441000,74.1,37966.0,36846.0,Al-Imarat al-??Arabiya al-Muttahida,Emirate Federation,Zayid bin Sultan al-Nahayan,65.0,AE
3,ARM,Armenia,Asia,Middle East,29800.0,1991.0,3520000,66.4,1813.0,1627.0,Hajastan,Republic,Robert Kot??arjan,126.0,AM
4,AZE,Azerbaijan,Asia,Middle East,86600.0,1991.0,7734000,62.9,4127.0,4100.0,Az??rbaycan,Federal Republic,Heyd??r ??liyev,144.0,AZ


利用子查詢與UPDATE更新資料:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,11952.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
1,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,13296.0,7984.0,Angola,Republic,Jos?? Eduardo dos Santos,56.0,AO
2,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971.0,2441000,74.1,37966.0,36846.0,Al-Imarat al-??Arabiya al-Muttahida,Emirate Federation,Zayid bin Sultan al-Nahayan,65.0,AE
3,ARM,Armenia,Asia,Middle East,29800.0,1991.0,3520000,66.4,3626.0,1627.0,Hajastan,Republic,Robert Kot??arjan,126.0,AM
4,AZE,Azerbaijan,Asia,Middle East,86600.0,1991.0,7734000,62.9,8254.0,4100.0,Az??rbaycan,Federal Republic,Heyd??r ??liyev,144.0,AZ


### 刪除資料 (DELETE FROM, TRUNCATE)
刪除表格中不再需要的紀錄使用**「DELETE」**敘述，使用「DELETE」敘述的時候，通常也會使用**「WHERE」**、**「ORDER BY」**與**「LIMIT」**子句設定要刪除哪些紀錄。

如果要刪除一個表格中所有的紀錄，你可以選擇使用**「TRUNCATE」**敘述，「TRUNCATE」敘述在執行刪除紀錄的時候，會比使用「DELETE」敘述的**效率好**一些，尤其是表格中的紀錄非常多的時候會更明顯。而「TRUNCATE TABLE」與「DROP TABLE」的不同之處在於使用「TRUNCATE TABLE」時仍會保留表格的資料結構以供未來使用，「DROP TABLE」則會將一切刪除。

In [48]:
cursor.execute("USE cmdev")

# 使用 DELETE 刪除資料
print("使用DELETE刪除資料:")
cursor.execute("DELETE FROM travel_copy \
                WHERE location = 'DALLAS' \
                ORDER BY empno \
                LIMIT 1")
cursor.execute("SELECT * FROM travel_copy")
showdata(cursor)

# 使用 TRUNCATE 刪除表格
print("使用TRUNCATE刪除表格:")
cursor.execute("TRUNCATE TABLE travel_copy")
cursor.execute("SELECT * FROM travel_copy")
showdata(cursor)

使用DELETE刪除資料:


,empno,location,counter
0,7369,CHICAGO,10
1,7521,DALLAS,2
2,7566,BOSTON,2
3,7654,NEW YORK,2
4,7777,DALLAS,0


使用TRUNCATE刪除表格:


""


#### 刪除資料與子查詢
使用「DELETE」敘述執行修改資料時，如果沒有使用「WHERE」子句指定刪除的條件，「DELETE」敘述會**刪除表格中所有的紀錄**。所以刪除資料的時候，通常會使用「WHERE」子句指定刪除的條件。在「DELETE」敘述的「WHERE」子句，也可以使用子查詢提供判斷條件的資料。

In [49]:
cursor.execute("USE world")

# 刪除資料前
print("刪除資料前:")
cursor.execute("SELECT * FROM country_copy")
showdata(cursor, header=5)

# 利用子查詢與 DELETE 刪除資料
print("利用子查詢與DELETE刪除資料:")
cursor.execute("DELETE FROM country_copy \
                WHERE Code IN (SELECT CountryCode \
                               FROM city \
                               WHERE Population > 1000000)")
cursor.execute("SELECT * FROM country_copy")
showdata(cursor, header=5)

刪除資料前:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,11952.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
1,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,13296.0,7984.0,Angola,Republic,Jos?? Eduardo dos Santos,56.0,AO
2,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971.0,2441000,74.1,37966.0,36846.0,Al-Imarat al-??Arabiya al-Muttahida,Emirate Federation,Zayid bin Sultan al-Nahayan,65.0,AE
3,ARM,Armenia,Asia,Middle East,29800.0,1991.0,3520000,66.4,3626.0,1627.0,Hajastan,Republic,Robert Kot??arjan,126.0,AM
4,AZE,Azerbaijan,Asia,Middle East,86600.0,1991.0,7734000,62.9,8254.0,4100.0,Az??rbaycan,Federal Republic,Heyd??r ??liyev,144.0,AZ


利用子查詢與DELETE刪除資料:


,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971.0,2441000,74.1,37966.0,36846.0,Al-Imarat al-??Arabiya al-Muttahida,Emirate Federation,Zayid bin Sultan al-Nahayan,65.0,AE
1,BDI,Burundi,Africa,Eastern Africa,27834.0,1962.0,6695000,46.2,903.0,982.0,Burundi/Uburundi,Republic,Pierre Buyoya,552.0,BI
2,BEN,Benin,Africa,Western Africa,112622.0,1960.0,6097000,50.2,2357.0,2141.0,B??nin,Republic,Mathieu K??r??kou,187.0,BJ
3,BFA,Burkina Faso,Africa,Western Africa,274000.0,1960.0,11937000,46.7,2425.0,2201.0,Burkina Faso,Republic,Blaise Compaor??,549.0,BF
4,BHR,Bahrain,Asia,Middle East,694.0,1971.0,617000,73.0,6366.0,6097.0,Al-Bahrayn,Monarchy (Emirate),Hamad ibn Isa al-Khalifa,149.0,BH


# 資料匯入與匯出
在開始使用 MySQL資料庫以後，MySQL 會儲存與管理所有的資料，依照不同的設定，會有許多的資料檔案儲存在檔案系統中，如果這些檔案不小
心遺失或損壞，儲存的資料可能就全部不見了。為了預防這類的情況發生，MySQL 提供許多備份資料的功能，讓你可以依照自己的需求，匯出資料庫中儲存的資料，另外保存起來。如果資料庫發生嚴重的問題，而且儲存的資料不見了，你就可以把之前備份的資料回復到資料庫中。備份資料的工作稱為「匯出資料、exporting data」，回復資料的工作稱為「匯入資料、importing data 」。

## 資料匯出(SELECT INTO OUTFILE)
在MySQL中可以使用一些敘述子句來修改匯出格式，如以「FIELDS TERMINATED BY 」子句設定新的分隔字元，「FIELDS ENCLOSED BY 」可以設定包圍欄位資料的字元符號，「FIELDS ESCAPED BY 」子句設定新的跳脫字元符號，而使用「LINES STARTING BY」與「TERMINATED BY」 子句可以設定每一列資料開始與結束字串。

In [50]:
cursor.execute("USE cmdev")

# 匯出資料
try:
    cursor.execute("SELECT * INTO OUTFILE 'C:/ProgramData/MySQL/MySQL Server 5.7/Data/cmdev_emp.txt' \
                    FIELDS TERMINATED BY ',' ESCAPED BY '$' ENCLOSED BY '\"' \
                    LINES STARTING BY '>> ' TERMINATED BY '\r' \
                    FROM cmdev.emp \
                    WHERE salary < 1500")
    
except:
    pass
    
finally:
    with open('C:/ProgramData/MySQL/MySQL Server 5.7/Data/cmdev_emp.txt', 'r') as f:
        print(f.read())

>> "7369","SMITH","CLERK","7902","1980-12-17","800.00",$N,"20"
>> "7521","WARD","SALESMAN","7698","1981-02-22","1250.00","500.00","30"
>> "7654","MARTIN","SALESMAN","7698","1981-09-28","1250.00","1400.00","30"
>> "7876","ADAMS","CLERK","7788","1987-05-23","1100.00",$N,"20"
>> "7900","JAMES","CLERK","7698","1981-12-03","950.00",$N,$N
>> "7934","MILLER","CLERK","7782","1982-01-23","1300.00",$N,"10"



## 資料匯入 (LOAD DATA INFILE)
「LOAD DATA INFILE」敘述可以將一個包含資料的檔案，匯入到一個指定的表格中，而在新增、修改或匯入資料到資料庫的時候，都有可能發生索引值重複的錯誤，如果發生此種情況，可以使用「IGNORE」或「REPLACE」來決定資料庫該作什麼處理。資料檔儲存在 MySQL 伺服器的電腦的時候，可以使用「IGNORE」關鍵字忽略錯誤的資料，正確的資料還是匯入到資料庫中；使用「REPLACE」關鍵字則會請資料庫會幫你執行修改資料的動作。

In [51]:
# 使用查詢表格來建立 cmdev.emp 的複製品
cursor.execute("CREATE TABLE IF NOT EXISTS emp_copy2( \
                PRIMARY KEY (empno)) \
                SELECT * \
                FROM cmdev.emp")

In [52]:
# 匯入資料
cursor.execute("LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 5.7/Data/cmdev_emp.txt' \
                IGNORE INTO TABLE cmdev.emp_copy2 \
                FIELDS TERMINATED BY ',' ESCAPED BY '$' ENCLOSED BY '\"' \
                LINES STARTING BY '>> ' TERMINATED BY '\r'")

cursor.execute("SELECT * FROM cmdev.emp_copy2")
showdata(cursor)

,empno,ename,job,manager,hiredate,salary,comm,deptno
0,7369,SMITH,CLERK,7902,1980-12-17,800.0,NaN,20.0
1,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.0,30.0
2,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.0,30.0
3,7876,ADAMS,CLERK,7788,1987-05-23,1100.0,NaN,20.0
4,7900,JAMES,CLERK,7698,1981-12-03,950.0,NaN,NaN
5,7934,MILLER,CLERK,7782,1982-01-23,1300.0,NaN,10.0


# View元件
資料庫在運作一段時間以後，不論是查詢與維護的敘述，都可能會出現一些類似，而且很常使用的SQL敘述。如果在資料庫的應用中，出現這種很常執行的查詢敘述，可以在MySQL資料庫中建立一種**「View」元件**，「View」元件用來**保存一個設定好的查詢敘述**。「View」元件也可以稱為「虛擬表格」，除了有一些限制外，它使用起來就像是一個表格一樣以查詢敘述的「FROM」子句開頭，也可以使用 View 元件回傳的紀錄資料，執行統計、分組與其它需要的處理。


## 建立View元件 (CREATE OR REPLACE VIEW)
不論是執行查詢或維護工作，如果很常需要使用到同一個查詢敘述，就可以考慮使用**「CREATE VIEW」**建立一個「View」元件把這個查詢敘述儲存起來。如果需要修改一個已經建立好的「View」元件，就要加入**「OR REPLACE」**的設定，這樣才不會出現錯誤訊息。

下列是MySQL關於「View」元件的規定與限制:
+ 在同一個資料庫中，「View」元件的名稱不可以重複，也不可以跟表格名稱一樣
+ 「View」元件不可以跟「Triggers」建立聯結

儲存在 View 元件中的查詢敘述有下列的規定:
+ 查詢敘述中只能使用到已存在的表格或「View」元件
+ 「FROM」子句中不可以使用子查詢
+ 不可以使用「TEMPORARY」表格
+ 不可以使用自定義的變數、「Procedure」與「Prepare statement」參數

In [53]:
cursor.execute("USE world")

# 建立 View
print("建立View:")
cursor.execute("CREATE OR REPLACE VIEW CountryMaxGNP AS \
                SELECT Name, GNP \
                FROM country \
                WHERE (Region, GNP) IN (SELECT Region, MAX(GNP) \
                                        FROM country \
                                        GROUP BY Region)")
cursor.execute("SELECT * FROM CountryMaxGNP")
showdata(cursor, header=10)

# 建立 View 時有相同欄位名稱要重新命不同的名稱
print("建立View時重新命名:")
cursor.execute("CREATE OR REPLACE VIEW ScaleView (CountryName, CityName, CountryPop, CityPop, Scale) AS \
                SELECT country.name, city.Name, country.Population, city.Population, ROUND (city.Population / country.Population, 2) Scale \
                FROM country, city \
                WHERE country.Code = city.CountryCode")
cursor.execute("SELECT * FROM ScaleView")
showdata(cursor, header=10)

建立View:


,Name,GNP
0,Antarctica,0.0
1,French Southern territories,0.0
2,Australia,351182.0
3,Brazil,776739.0
4,Bouvet Island,0.0
5,Cameroon,9174.0
6,Germany,2133367.0
7,Egypt,82710.0
8,United Kingdom,1378330.0
9,Guam,1197.0


建立View時重新命名:


,CountryName,CityName,CountryPop,CityPop,Scale
0,Afghanistan,Kabul,22720000,1780000,0.08
1,Afghanistan,Qandahar,22720000,237500,0.01
2,Afghanistan,Herat,22720000,186800,0.01
3,Afghanistan,Mazar-e-Sharif,22720000,127800,0.01
4,Netherlands,Amsterdam,15864000,731200,0.05
5,Netherlands,Rotterdam,15864000,593321,0.04
6,Netherlands,Haag,15864000,440900,0.03
7,Netherlands,Utrecht,15864000,234323,0.01
8,Netherlands,Eindhoven,15864000,201843,0.01
9,Netherlands,Tilburg,15864000,193238,0.01


## 修改View元件 (ALTER VIEW)
使用「ALTER VIEW」敘述可以修改一個已經建立好的「View」元件，以執行修改「View」元件的工作來說,使用「ALTER VIEW」或「CREATE OR
REPLACE VIEW」敘述的效果是完全一樣的，唯一的差異是要修改的「View」元件如果不存在，「CREATE OR REPLACE VIEW」敘述會直接建立新的「View」元件。

In [54]:
# 修改 View 前
print("修改View前:")
cursor.execute("SELECT * FROM CountryMaxGNP")
showdata(cursor, header=10)

# 修改 View 後
print("修改View後:")
cursor.execute("ALTER VIEW CountryMaxGNP AS \
                SELECT Continent, Name, GNP \
                FROM country \
                WHERE (Region, GNP) IN (SELECT Region, MAX(GNP) \
                                        FROM country \
                                        GROUP BY Region)")
cursor.execute("SELECT * FROM CountryMaxGNP")
showdata(cursor, header=10)

修改View前:


,Name,GNP
0,Antarctica,0.0
1,French Southern territories,0.0
2,Australia,351182.0
3,Brazil,776739.0
4,Bouvet Island,0.0
5,Cameroon,9174.0
6,Germany,2133367.0
7,Egypt,82710.0
8,United Kingdom,1378330.0
9,Guam,1197.0


修改View後:


,Continent,Name,GNP
0,Antarctica,Antarctica,0.0
1,Antarctica,French Southern territories,0.0
2,Oceania,Australia,351182.0
3,South America,Brazil,776739.0
4,Antarctica,Bouvet Island,0.0
5,Africa,Cameroon,9174.0
6,Europe,Germany,2133367.0
7,Africa,Egypt,82710.0
8,Europe,United Kingdom,1378330.0
9,Oceania,Guam,1197.0


## 刪除View元件 (DROP VIEW)
使用「DROP VIEW」敘述可以刪除一個建立好的「View」元件，若指定的「View」元件不存在則會產生錯誤訊息，所以可以加入「IF EXISTS」敘述避免錯誤訊息。

In [55]:
# 刪除 View 前
print("刪除View前:")
cursor.execute("SHOW FULL TABLES IN world WHERE TABLE_TYPE LIKE 'VIEW'")
showdata(cursor, header=10)




# 刪除 View 後
print("刪除View後:")
cursor.execute("DROP VIEW IF EXISTS CountryMaxGNP")
cursor.execute("SHOW FULL TABLES IN world WHERE TABLE_TYPE LIKE 'VIEW'")
showdata(cursor, header='all')

刪除View前:


,Tables_in_world,Table_type
0,countrymaxgnp,VIEW
1,scaleview,VIEW


刪除View後:


,Tables_in_world,Table_type
0,scaleview,VIEW


## View元件的資料維護 (INSERT INTO, UPDATE, DELETE)
要使用「View」元件執行新增、修改或刪除資料的工作，「View」元件所包含的查詢敘述必須符合下列的規則:
+ 不可以包含計算或函式的欄位
+ 只允許一對一的結合查詢
+ 「View」元件的「ALGORITHM」不可以設定為「TEMPTABLE」

符合上列規定的「View」元件，就會稱為**可修改資料的「View」元件**、**updatable「View」**，只有可修改資料的「View」元件可以使用在「INSERT」、「UPDATE」或「DELETE」敘述。

在使用「View」元件執行資料維護的工作，可能會產生一些有問題的資料，如果不希望產生這類的問題，可以為「View」元件加入**「WITH CHECK OPTION」**的設定，如此一來，在執行資料維護工作時，MySQL會先執行檢查的工作，規則是**一定要符合「View」元件中「WHERE」設定的條件**。而「WITH CHECK OPTION」設定，有額外的**「CASCADED」**和**「LOCAL」**兩個控制檢查範圍的設定，會有「CASCADED」和「LOCAL」這兩個設定的原因，是因為「View」元件的資料來源可以是一個表格，也可以是另一個「View」元件，而檢查範圍設定為「LOCAL」在執行資料維護的時候，只會檢查是否**符合自己的條件設定**；檢查範圍設定為「CASCADED」在執行資料維護的時候，就**不能違反所有「View」元件的條件設定**。

In [56]:
cursor.execute("USE cmdev")

# 建立 View 並利用 WITH CASCADED CHECK OPTION 設定不能違反所有「View」元件的條件設定
print("建立View:")
cursor.execute("CREATE OR REPLACE VIEW EmpDept30View AS \
                SELECT empno, ename, job, manager, hiredate, salary, comm, deptno\
                FROM emp_copy \
                WHERE deptno = 30 \
                WITH CASCADED CHECK OPTION")
cursor.execute("SELECT * FROM EmpDept30View")
showdata(cursor)

# 新增 View 資料
print("新增View資料:")
cursor.execute("INSERT INTO EmpDept30View (empno, ename, job, manager, hiredate, salary, comm, deptno) \
                VALUES (9001, 'SIMON', 'SALESMAN', 7698, '2000-04-01', 1000, 250, 30)")
cursor.execute("SELECT * FROM EmpDept30View")
showdata(cursor)

# 新增不合規定 View 資料
print("新增不合規定View資料:")
try:
    cursor.execute("INSERT INTO EmpDept30View (empno, ename, job, manager, hiredate, salary, comm, deptno) \
                    VALUES (9002, 'BEN', 'SALESMAN', 7698, '2010-01-01', 1700, 350, 50)")
    cursor.execute("SELECT * FROM EmpDept30View")
    showdata(cursor)
except:
    print("違反EmpDept30View的設定\n")

# 更新 View 資料
print("更新View資料:")
cursor.execute("UPDATE EmpDept30View \
                SET comm = 600 \
                WHERE ename = 'TURNER'")
cursor.execute("SELECT * FROM EmpDept30View")
showdata(cursor)

# 刪除 View 資料
print("刪除View資料:")
cursor.execute("DELETE FROM EmpDept30View \
                WHERE ename = 'ALLEN'")
cursor.execute("SELECT * FROM EmpDept30View")
showdata(cursor)

# 原表格也被更改
print("原表格emp:")
cursor.execute("SELECT empno, ename, job, manager, hiredate, salary, comm, deptno \
                FROM emp_copy \
                WHERE job = 'SALESMAN'")
showdata(cursor)

建立View:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.0,30
1,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.0,30
2,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.0,30
3,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,0.0,30


新增View資料:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.0,30
1,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.0,30
2,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.0,30
3,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,0.0,30
4,9001,SIMON,SALESMAN,7698,2000-04-01,1000.0,250.0,30


新增不合規定View資料:
違反EmpDept30View的設定

更新View資料:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.0,300.0,30
1,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.0,30
2,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.0,30
3,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,600.0,30
4,9001,SIMON,SALESMAN,7698,2000-04-01,1000.0,250.0,30


刪除View資料:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.0,30
1,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.0,30
2,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,600.0,30
3,9001,SIMON,SALESMAN,7698,2000-04-01,1000.0,250.0,30


原表格emp:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.0,30
1,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.0,30
2,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,600.0,30
3,9001,SIMON,SALESMAN,7698,2000-04-01,1000.0,250.0,30


## 取得View元件的資訊 (CHECK TABLE)
在建立一個「View」元件的時候，MySQL 會檢查「View」元件包含的查詢敘述是否正確，如果沒有問題的話，才會儲存「View」元件的設定。MySQL資料庫伺服器在啟動以後，有一個名稱為**「information_schema」**的資料庫，這個資料庫通常會稱為「系統資訊資料庫」。這個資料庫中有一個表格叫作**「VIEWS」** 它儲存所有 MySQL 資料庫中「View」元件的相關資訊。

In [57]:
# 建立兩個有關聯的 View 
cursor.execute("CREATE OR REPLACE VIEW EmpSalaryView AS \
                SELECT * \
                FROM emp_copy \
                WHERE salary >= 1500")
cursor.execute("CREATE OR REPLACE VIEW EmpDept20View AS \
                SELECT * \
                FROM EmpSalaryView \
                WHERE deptno = 20 \
                WITH LOCAL CHECK OPTION")

# 檢查 View 的可靠性
print("檢查View的可靠性:")
cursor.execute("CHECK TABLE EmpDept20View")
showdata(cursor)

# 刪除 View
print("已刪除EmpSalaryView\n")
cursor.execute("DROP VIEW IF EXISTS EmpSalaryView")

# 再次檢查 View 的可靠性
print("再次檢查 View 的可靠性:")
cursor.execute("CHECK TABLE EmpDept20View")
showdata(cursor)

# 取得 world, cmdev 資料庫內所有 View 的資訊
print("取得所有 world, cmdev 資料庫內所有 View 的資訊:")
cursor.execute("SELECT TABLE_SCHEMA, TABLE_NAME, VIEW_DEFINITION, CHECK_OPTION, IS_UPDATABLE \
                FROM information_schema.VIEWS \
                WHERE TABLE_SCHEMA IN ('world', 'cmdev')")
showdata(cursor)

檢查View的可靠性:


,Table,Op,Msg_type,Msg_text
0,cmdev.empdept20view,check,status,OK


已刪除EmpSalaryView

再次檢查 View 的可靠性:


,Table,Op,Msg_type,Msg_text
0,cmdev.empdept20view,check,Error,Table 'cmdev.empsalaryview' doesn't exist
1,cmdev.empdept20view,check,Error,View 'cmdev.empdept20view' references invalid ...
2,cmdev.empdept20view,check,error,Corrupt


取得所有 world, cmdev 資料庫內所有 View 的資訊:


,TABLE_SCHEMA,TABLE_NAME,VIEW_DEFINITION,CHECK_OPTION,IS_UPDATABLE
0,cmdev,empdept20view,"select `empsalaryview`.`empno` AS `empno`,`emp...",LOCAL,
1,cmdev,empdept30view,"select `cmdev`.`emp_copy`.`empno` AS `empno`,`...",CASCADED,YES
2,world,scaleview,select `world`.`country`.`Name` AS `CountryNam...,NONE,YES


# Prepared Statements
一個資料庫在建立好並開始使用以後，資料庫伺服器就會接收各種不同的敘述來執行工作。以查詢敘述來說，有一些敘述可能大部份的內容都是一樣的，只有在條件的設定上會不一樣。就算這些敘述的內容是差不多的，資料庫伺服器每次接收到敘述時，還是要依照流程執行一些同樣的工作。

伺服器已經準備好的敘述就稱為「prepared statement」，後續要使用這種敘述前，要先設定好「prepared statement」需要的資料。然後再請伺服器執行指定的「prepared statement」，伺服器就會傳回執行後的結果了。

## User Variables (SET @var.)
MySQL資料庫伺服器提供一種簡易的儲存資料方式，稱為**「user variables」**。使用者變數儲存一些簡單的資料，例如數字或字串，它們可以在後續的操作中使用。而設定使用者變數可以使用**「SET」**。

In [58]:
# 使用 SET 建立多個使用者變數
print("使用SET建立多個使用者變數:")
cursor.execute("SET @user_job = 'SALESMAN', \
                    @user_job2 = 'MANAGER'")
cursor.execute("SELECT @user_job, @user_job2")
showdata(cursor)

# 使用變數做查詢
print("使用變數做查詢:")
cursor.execute("SELECT * \
                FROM emp_copy \
                WHERE job IN (@user_job, @user_job2)")
showdata(cursor)

使用SET建立多個使用者變數:


,@user_job,@user_job2
0,SALESMAN,MANAGER


使用變數做查詢:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,7521,WARD,SALESMAN,7698,1981-02-22,1250.0,500.0,30.0
1,7566,JONES,MANAGER,7839,1981-04-02,2975.0,NaN,20.0
2,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.0,1400.0,30.0
3,7698,BLAKE,MANAGER,7839,1981-05-01,2850.0,NaN,NaN
4,7782,CLARK,MANAGER,7839,1981-06-09,2450.0,NaN,10.0
5,7844,TURNER,SALESMAN,7698,1981-09-08,1500.0,600.0,30.0
6,8001,SIMON,MANAGER,7369,2001-02-03,3300.0,NaN,50.0
7,9001,SIMON,SALESMAN,7698,2000-04-01,1000.0,250.0,30.0


## 建立Prepared Statements (PREPARE FROM)
在建立「prepared statement」時,你會依照敘述的需求設定參數標記,這些參數標記也決定執行「prepared statement」時,需要**照順序傳送多少參數資料給它才可以正確的執行**。

In [59]:
# 建立 Prepared Statements
cursor.execute("PREPARE user_emp FROM \
                'INSERT INTO emp_copy VALUES (?, ?, ?, ?, ?, ?, ?, ?)'")

## 執行Prepared Statements (EXECUTE USING)


In [60]:
# 使用 SET 建立多個使用者變數
print("使用SET建立多個使用者變數:")
cursor.execute("SET @user_empno = 0001, \
                    @user_ename = 'MARDI', \
                    @user_job = 'BOSS', \
                    @user_manager = NULL, \
                    @user_hiredate = '2010-01-01', \
                    @user_salary = 50000, \
                    @user_comm = 77777, \
                    @user_deptno = 99")
cursor.execute("SELECT @user_empno, @user_ename, @user_job, @user_manager, \
                       @user_hiredate, @user_salary, @user_comm, @user_deptno")
showdata(cursor)

# 執行 Prepared Statements
print("執行Prepared Statements:")
cursor.execute("EXECUTE user_emp USING @user_empno, \
                                       @user_ename, \
                                       @user_job, \
                                       @user_manager, \
                                       @user_hiredate, \
                                       @user_salary, \
                                       @user_comm, \
                                       @user_deptno")
cursor.execute("SELECT * FROM emp_copy")
showdata(cursor, 'all')

使用SET建立多個使用者變數:


,@user_empno,@user_ename,@user_job,@user_manager,@user_hiredate,@user_salary,@user_comm,@user_deptno
0,1,MARDI,BOSS,None,2010-01-01,50000,77777,99


執行Prepared Statements:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,1,MARDI,BOSS,NaN,2010-01-01,50000.0,77777.0,99.0
1,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20.0
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20.0
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,NaN
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10.0
7,7788,SCOTT,ANALYST,7566.0,1987-04-19,3000.0,NaN,20.0
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10.0
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,600.0,30.0


## 刪除Prepared Statements (DROP)

In [61]:
# 刪除 Prepared Statements
try:
    cursor.execute("DROP PREPARE user_emp")
    print("已刪除Prepared Statements")
    
except:
    pass

已刪除Prepared Statements


# Stored Routines


## Stored Procedures
SQL敘述的特點是一次只能執行一件工作，所以要完成一些比較複雜的需求，就必須執行好幾個SQL敘述。如果經常執行這一組工作，就可以考慮把這些要執行的敘述建立為「Stored procedure」元件。

「Stored procedure」元件是一種可以建立、維護與刪除的資料庫元件。表格元件是用來儲存資料用的，索引元件是儲存索引與增加效率用的，而 「Stored procedure」元件是用來「儲存程序」用的。程序表示一組特定的工作，如果在使用資料庫的過程中，**經常需要執行一組同樣的工作**，你就可以考慮把執行工作需要的敘述建立為「Stored procedure」元件。

procedure 參數用途的說明:
+ IN: 輸入用的參數，在呼叫 procedure 時傳送資料給 procedure 使用。
+ OUT: 輸出用的參數，在呼叫 procedure 時，不能接收傳送的資料，不過 procedure 在執行的時候，可以設定這種參數的值，新的值在執行完成後，可以回傳給呼叫的地方使用。
+ INOUT: 輸入與輸出用的參數，同時具有「IN」與「OUT」兩種用途。

### 建立Stored Procedures (CREATE PROCEDURE)

In [62]:
cursor.execute("USE world")

# 建立 Stored Procedure
print("建立Stored Procedure:")
cursor.execute("CREATE PROCEDURE country_count(IN cont VARCHAR(30), OUT count INT) \
                BEGIN \
                    SELECT COUNT(*) INTO count \
                    FROM country \
                    WHERE Continent = cont; \
                END")
cursor.execute("SHOW PROCEDURE STATUS \
                WHERE db='world'")
showdata(cursor)

建立Stored Procedure:


,Db,Name,Type,Definer,Modified,Created,Security_type,Comment,character_set_client,collation_connection,Database Collation
0,world,country_count,PROCEDURE,root@localhost,2020-03-29 16:50:49,2020-03-29 16:50:49,DEFINER,,utf8mb4,utf8mb4_general_ci,big5_chinese_ci


### 執行Stored Procedures (CALL)

In [63]:
cursor.execute("SET @cont = 'Asia'")
cursor.execute("CALL country_count(@cont, @count)")
cursor.execute("SELECT @count")
showdata(cursor)

,@count
0,51


### 刪除Stored Procedures (DROP PROCEDURE)

In [64]:
# 刪除 Stored Procedure
print("刪除Stored Procedure:")
cursor.execute("DROP PROCEDURE IF EXISTS country_count")
cursor.execute("SHOW PROCEDURE STATUS \
                WHERE db='world'")
showdata(cursor)

刪除Stored Procedure:


""


## Stored Functions
如果 MySQL 提供的函式無法完成你的工作，或是想要改善一些比較複雜的工作，你可以自己建立需要的「Stored functions」元件。跟「Stored procedure」一樣，它也是一種用來**儲存程序的元件**，不過建立好的「Stored procedure」要使用「CALL」來呼叫，也就是請資料庫執行儲存在 「Stored procedure」元件中的工作，而要使用建立好的「Stored functions」元件，就**跟使用 MySQL 提供的函式一樣的用法**。

使用**「CREATE FUNCTION」**語法建立 「Stored functions 元件」，其中包含**「RETURNS」**與**「RETURN」**兩個關鍵字。

### 建立Stored Functions (CREATE FUNCTION)

In [65]:
# 建立 Stored Function
print("建立Stored Function:")
cursor.execute("CREATE FUNCTION TWO_SUM(var1 INT, var2 INT) \
                RETURNS INT \
                BEGIN \
                    RETURN var1 + var2; \
                END")
cursor.execute("SHOW FUNCTION STATUS \
                WHERE db='world'")
showdata(cursor)

建立Stored Function:


,Db,Name,Type,Definer,Modified,Created,Security_type,Comment,character_set_client,collation_connection,Database Collation
0,world,TWO_SUM,FUNCTION,root@localhost,2020-03-29 16:50:50,2020-03-29 16:50:50,DEFINER,,utf8mb4,utf8mb4_general_ci,big5_chinese_ci


### 執行Stored Functions

In [66]:
# 執行 Stored Function
print("執行Stored Function:")
cursor.execute("SET @var1 = 2, @var2 = 5")
cursor.execute("SELECT TWO_SUM(@var1, @var2)")
showdata(cursor)

執行Stored Function:


,"TWO_SUM(@var1, @var2)"
0,7


### 刪除Drop Functions (DROP FUNCTION)

In [67]:
# 刪除 Stored Function
print("刪除Stored Function:")
cursor.execute("DROP FUNCTION IF EXISTS TWO_SUM")
cursor.execute("SHOW FUNCTION STATUS \
                WHERE db='world'")
showdata(cursor)

刪除Stored Function:


""


## Local Variables (DECLARE DEFAULT)
在「Stored routines」中可以使用**「DECALRE」**宣告**「區域變數」(local variables)**，宣告需要的區域變數以後就可以使用它們，而需要指定變數值的話，可以使用**SET**以及**SELECT INTO**。在一個「Stored routines」中，除了基本的 BEGIN-END 區塊，也可以再使用 BEGIN-END 設定額外的區塊，並且在每一個區都可以宣告需要的區域變數。

In [68]:
cursor.execute("CREATE FUNCTION TOT_COUNTRYCITY() \
                RETURNS INT \
                BEGIN \
                    DECLARE country_no, city_no INT; \
                    DECLARE total_no INT DEFAULT 0; \
                    \
                    SELECT COUNT(*) INTO country_no FROM country; \
                    SELECT COUNT(*) INTO city_no FROM city; \
                    SET total_no = country_no + city_no; \
                    \
                    RETURN total_no; \
                END")

cursor.execute("SELECT TOT_COUNTRYCITY()")
showdata(cursor)

cursor.execute("DROP FUNCTION IF EXISTS TOT_COUNTRYCITY")

,TOT_COUNTRYCITY()
0,4318


## Conditional Processing (IF, CASE)

In [69]:
# IF 控制流程
print("IF控制流程:")
cursor.execute("CREATE FUNCTION GET_SEASON_IF(month INT) \
                RETURNS VARCHAR(20) \
                BEGIN \
                    DECLARE season VARCHAR(20); \
                    \
                    IF month BETWEEN 1 AND 3 THEN \
                        SET season = 'Spring'; \
                    ELSEIF month BETWEEN 4 AND 6 THEN \
                        SET season = 'Summer'; \
                    ELSEIF month BETWEEN 7 AND 9 THEN \
                        SET season = 'Fall'; \
                    ELSEIF month BETWEEN 10 AND 12 THEN \
                        SET season = 'Winter'; \
                    ELSE \
                        SET season = 'No this month'; \
                    END IF; \
                    \
                    RETURN season; \
                END")

cursor.execute("SELECT GET_SEASON_IF(2), GET_SEASON_IF(5), GET_SEASON_IF(9), GET_SEASON_IF(11), GET_SEASON_IF(13)")
showdata(cursor)

cursor.execute("DROP FUNCTION IF EXISTS GET_SEASON_IF")

IF控制流程:


,GET_SEASON_IF(2),GET_SEASON_IF(5),GET_SEASON_IF(9),GET_SEASON_IF(11),GET_SEASON_IF(13)
0,Spring,Summer,Fall,Winter,No this month


In [70]:
# CASE 控制流程
print("CASE控制流程:")
cursor.execute("CREATE FUNCTION GET_SEASON_CASE(qauter INT) \
                RETURNS VARCHAR(20) \
                BEGIN \
                    DECLARE season VARCHAR(20); \
                    \
                    CASE qauter \
                    WHEN 1 THEN SET season = 'Spring'; \
                    WHEN 2 THEN SET season = 'Summer'; \
                    WHEN 3 THEN SET season = 'Fall'; \
                    WHEN 4 THEN SET season = 'Winter'; \
                    ELSE SET season = 'No this qauter'; \
                    END CASE; \
                    \
                    RETURN season; \
                END")

cursor.execute("SELECT GET_SEASON_CASE(1), GET_SEASON_CASE(2), GET_SEASON_CASE(3), GET_SEASON_CASE(4), GET_SEASON_CASE(5)")
showdata(cursor)

cursor.execute("DROP FUNCTION IF EXISTS GET_SEASON_CASE")

CASE控制流程:


,GET_SEASON_CASE(1),GET_SEASON_CASE(2),GET_SEASON_CASE(3),GET_SEASON_CASE(4),GET_SEASON_CASE(5)
0,Spring,Summer,Fall,Winter,No this qauter


## Loop (WHILE, REPEAT, LOOP, LEAVE, ITERATE)

In [71]:
# WHILE 迴圈
print("WHILE迴圈:")
cursor.execute("CREATE FUNCTION SUM_FROM1TON_WHILE(N INT) \
                RETURNS INT \
                BEGIN \
                    DECLARE i INT DEFAULT 1; \
                    DECLARE tot INT DEFAULT 0; \
                    \
                    WHILE i <= N DO \
                        IF MOD(i, 2) = 0 THEN \
                            SET tot = tot + i; \
                        END IF; \
                        SET i = i + 1; \
                    END WHILE; \
                    \
                    RETURN tot; \
                END")

cursor.execute("SELECT SUM_FROM1TON_WHILE(10)")
showdata(cursor)

cursor.execute("DROP FUNCTION IF EXISTS SUM_FROM1TON_WHILE")

WHILE迴圈:


,SUM_FROM1TON_WHILE(10)
0,30


In [72]:
# REPEAT 迴圈
print("REPEAT迴圈:")
cursor.execute("CREATE FUNCTION SUM_FROM1TON_REPEAT(N INT) \
                RETURNS INT \
                BEGIN \
                    DECLARE i INT DEFAULT 1; \
                    DECLARE tot INT DEFAULT 0; \
                    \
                    REPEAT \
                        IF MOD(i, 2) = 0 THEN \
                            SET tot = tot + i; \
                        END IF; \
                        SET i = i + 1; \
                        UNTIL i > N \
                    END REPEAT; \
                    \
                    RETURN tot; \
                END")

cursor.execute("SELECT SUM_FROM1TON_REPEAT(10)")
showdata(cursor)

cursor.execute("DROP FUNCTION IF EXISTS SUM_FROM1TON_REPEAT")

REPEAT迴圈:


,SUM_FROM1TON_REPEAT(10)
0,30


In [73]:
# LOOP 迴圈
print("LOOP迴圈(需搭配LEAVE):")
cursor.execute("CREATE FUNCTION SUM_FROM1TON_LOOP(N INT) \
                RETURNS INT \
                BEGIN \
                    DECLARE i INT DEFAULT 1; \
                    DECLARE tot INT DEFAULT 0; \
                    \
                    label1: LOOP \
                        IF MOD(i, 2) = 0 THEN \
                            SET tot = tot + i; \
                        END IF; \
                        SET i = i + 1; \
                        \
                        IF i > N THEN \
                           LEAVE label1; \
                        END IF; \
                    END LOOP label1; \
                    \
                    RETURN tot; \
                END")

cursor.execute("SELECT SUM_FROM1TON_LOOP(10)")
showdata(cursor)

cursor.execute("DROP FUNCTION IF EXISTS SUM_FROM1TON_LOOP")

LOOP迴圈(需搭配LEAVE):


,SUM_FROM1TON_LOOP(10)
0,30


In [74]:
# 使用 ITERATE 跳回迴圈開頭處
print("使用ITERATE跳回迴圈開頭處:")
cursor.execute("CREATE FUNCTION SUM_FROM1TON_WHILE_ITR(N INT) \
                RETURNS INT \
                BEGIN \
                    DECLARE i INT DEFAULT 1; \
                    DECLARE tot INT DEFAULT 0; \
                    \
                    label1: WHILE i <= N DO \
                        SET i = i + 1; \
                        IF MOD(i, 2) = 1 THEN \
                            ITERATE label1; \
                        END IF; \
                        SET tot = tot + i; \
                    END WHILE label1; \
                    \
                    RETURN tot; \
                END")

cursor.execute("SELECT SUM_FROM1TON_WHILE_ITR(10)")
showdata(cursor)

cursor.execute("DROP FUNCTION IF EXISTS SUM_FROM1TON_WHILE_ITR")

使用ITERATE跳回迴圈開頭處:


,SUM_FROM1TON_WHILE_ITR(10)
0,30


# Triggers
Triggers 可以讓你先把一些在特定狀況要執行的敘述儲存起來，MySQL資料庫會在正確的時機自動幫你執行這些敘述。

## 建立Triggers (CREATE TRIGGER)
建立 Trigger 時可以依照自己的需求搭配以下敘述的用法，大體上來說就是 Trigger 誘發的時機點以及條件。此外還可以利用「OLD」與「NEW」取得修改前後的資訊。

| TIMING | EVENT                    |
|--------|--------------------------|
| BEFORE | INSERT / UPDATE / DELETE |
| AFTER  | INSERT / UPDATE / DELETE |


在建立 Trigger 的時候，要特別注意下列的限制:  
+ 同一個資料庫不可以有相同名稱的 Trigger
+ 「TEMPORARY 表格」與「View」不可以建立 Trigger
+ 不可以使用「SELECT」敘述
+ 不可以使用「CALL」敘述
+ 不可以使用「START TRANSACTION」、「COMMIT」與「ROLLBACK」

In [75]:
cursor.execute("USE cmdev")

# 建立 UPDATE TRIGGER
print("建立UPDATE TRIGGER:")
cursor.execute("CREATE TRIGGER emp_before_update BEFORE UPDATE ON emp_copy FOR EACH ROW \
                BEGIN \
                    DECLARE v_message VARCHAR(50) DEFAULT 'UPDATE: '; \
                    SET v_message = CONCAT(v_message, OLD.deptno, '->', NEW.deptno); \
                    \
                    IF OLD.deptno <> NEW.deptno THEN \
                        INSERT INTO emplog(message) VALUES (v_message); \
                    END IF; \
                END")

cursor.execute("SHOW TRIGGERS FROM cmdev")
showdata(cursor)

建立UPDATE TRIGGER:


,Trigger,Event,Table,Statement,Timing,Created,sql_mode,Definer,character_set_client,collation_connection,Database Collation
0,emp_before_update,UPDATE,emp_copy,BEGIN DECLARE v_message VA...,BEFORE,2020-03-29 16:50:51.590,"STRICT_TRANS_TABLES,NO_AUTO_CREATE_USER,NO_ENG...",root@localhost,utf8mb4,utf8mb4_general_ci,big5_chinese_ci


In [76]:
print("資料修改前:")
cursor.execute("SELECT * FROM emp_copy WHERE ename = 'MARDI'")
showdata(cursor)

print("資料修改後:")
cursor.execute("UPDATE emp_copy SET deptno = 666 WHERE ename = 'MARDI'")
cursor.execute("SELECT * FROM emp_copy WHERE ename = 'MARDI'")
showdata(cursor)

print("TRIGGER log:")
cursor.execute("SELECT * FROM emplog")
showdata(cursor)

cursor.execute("TRUNCATE TABLE emplog")

資料修改前:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,1,MARDI,BOSS,None,2010-01-01,50000.0,77777.0,99


資料修改後:


,empno,ename,job,manager,hiredate,salary,comm,deptno
0,1,MARDI,BOSS,None,2010-01-01,50000.0,77777.0,666


TRIGGER log:


,logno,logdt,message
0,1,2020-03-29 16:50:51,UPDATE: 99->666


## 刪除Triggers(DROP TRIGGER)
如果想要修改 Trigger 元件中的敘述,你要先刪除原來的 Trigger 元件，再建立新的 Trigger 元件。所以通常會在建立 Trigger 元件的敘述中，加入刪除 Trigger 元件的敘述。

In [77]:
# 刪除 TRIGGER
print("刪除TRIGGER:")
cursor.execute("DROP TRIGGER IF EXISTS emp_before_update")
cursor.execute("SHOW TRIGGERS FROM cmdev")
showdata(cursor)

刪除TRIGGER:


""


# 資料庫資訊(SHOW)
以下為常用的資料庫資訊查詢敘述：

| QUERY                                               | EXPLANATION                    |
|-----------------------------------------------------|--------------------------------|
| SHOW TABLES FROM db                                 | 回傳資料庫中所有表格名稱       |
| SHOW TABLE STATUS FROM db                           | 回傳資料庫中所有表格資訊       |
| SHOW FUNCTION STATUS [LIKE 'pattern' \| WHERE expr]  | 回傳Function資訊               |
| SHOW PROCEDURE STATUS [LIKE 'pattern' \| WHERE expr] | 回傳Procedure資訊              |
| SHOW FULL COLUMNS FROM db.table                     | 回傳指定資料庫表格所有欄位資訊 |
| SHOW FULL TABLES IN db WHERE TABLE_TYPE LIKE 'VIEW' | 回傳指定資料庫中所有View資訊     |
| SHOW INDEX FROM db.table                            | 回傳指定資料庫表格中索引資訊   |
| SHOW TRIGGERS FROM db                               | 回傳指定資料庫中Trigger資訊    |

In [78]:
print("回傳資料庫中所有表格名稱:")
cursor.execute("SHOW TABLES FROM information_schema")
showdata(cursor, 5)

print("回傳資料庫中所有表格資訊:")
cursor.execute("SHOW TABLE STATUS FROM information_schema")
showdata(cursor, 5)

print("回傳指定資料庫表格所有欄位資訊:")
cursor.execute("SHOW FULL COLUMNS FROM cmdev.emp")
showdata(cursor)

print("回傳指定資料庫表格中索引資訊:")
cursor.execute("SHOW INDEX FROM cmdev.emp")
showdata(cursor)

print("回傳指定資料庫中Trigger資訊:")
cursor.execute("SHOW TRIGGERS FROM cmdev")
showdata(cursor)

回傳資料庫中所有表格名稱:


,Tables_in_information_schema
0,CHARACTER_SETS
1,COLLATIONS
2,COLLATION_CHARACTER_SET_APPLICABILITY
3,COLUMNS
4,COLUMN_PRIVILEGES


回傳資料庫中所有表格資訊:


,Name,Engine,Version,Row_format,Rows,Avg_row_length,Data_length,Max_data_length,Index_length,Data_free,Auto_increment,Create_time,Update_time,Check_time,Collation,Checksum,Create_options,Comment
0,CHARACTER_SETS,MEMORY,10,Fixed,None,384,0,14679936,0,0,None,2020-03-29 16:50:52,None,None,utf8_general_ci,None,max_rows=38229,
1,COLLATIONS,MEMORY,10,Fixed,None,231,0,14680050,0,0,None,2020-03-29 16:50:52,None,None,utf8_general_ci,None,max_rows=63550,
2,COLLATION_CHARACTER_SET_APPLICABILITY,MEMORY,10,Fixed,None,195,0,14679990,0,0,None,2020-03-29 16:50:52,None,None,utf8_general_ci,None,max_rows=75282,
3,COLUMNS,InnoDB,10,Dynamic,None,0,16384,0,0,8388608,None,NaT,None,None,utf8_general_ci,None,max_rows=2440,
4,COLUMN_PRIVILEGES,MEMORY,10,Fixed,None,2565,0,14679495,0,0,None,2020-03-29 16:50:52,None,None,utf8_general_ci,None,max_rows=5723,


回傳指定資料庫表格所有欄位資訊:


,Field,Type,Collation,Null,Key,Default,Extra,Privileges,Comment
0,empno,int(11),None,NO,PRI,None,,"select,insert,update,references",
1,ename,varchar(16),big5_chinese_ci,NO,,None,,"select,insert,update,references",
2,job,varchar(16),big5_chinese_ci,YES,,None,,"select,insert,update,references",
3,manager,int(11),None,YES,,None,,"select,insert,update,references",
4,hiredate,date,None,YES,,None,,"select,insert,update,references",
5,salary,"float(7,2)",None,YES,,None,,"select,insert,update,references",
6,comm,"float(7,2)",None,YES,,None,,"select,insert,update,references",
7,deptno,int(11),None,YES,,None,,"select,insert,update,references",


回傳指定資料庫表格中索引資訊:


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment
0,emp,0,PRIMARY,1,empno,A,14,None,None,,BTREE,,


回傳指定資料庫中Trigger資訊:


""


# 常用函式

## 字串

| FUNCTIONS                      | RETURN | EXPLANATION                                                             |
|--------------------------------|--------|-------------------------------------------------------------------------|
| LOWER(字串)                    | 字串   | 將[字串]轉換為小寫                                                      |
| UPPER(字串)                    | 字串   | 將[字串]轉換為大寫                                                      |
| LPAD(字串1,長度,字串2)         | 字串   | 如果[字串1]的長度小於指定的[長度]，就在[字串1]左邊使用[字串2]補滿       |
| RPAD(字串1,長度,字串2)         | 字串   | 如果[字串1]的長度小於指定的[長度]，就在[字串1]右邊使用[字串2]補滿       |
| LTRIM(字串)                    | 字串   | 移除[字串]左邊的空白                                                    |
| RTRIM(字串)                    | 字串   | 移除[字串]右邊的空白                                                    |
| TRIM(字串)                     | 字串   | 移除[字串]左、右的空白                                                  |
| REPEAT(字串,個數)              | 字串   | 重複[字串]指定的[個數]                                                  |
| REPLACE(字串1,字串2,字串3)     | 字串   | 將[字串1]中的[字串2]替換為[字串3]                                       |
| LEFT(字串,長度)                | 字串   | 傳回[字串]左邊指定[長度]的內容                                          |
| RIGHT(字串,長度)               | 字串   | 傳回[字串]右邊指定[長度]的內容                                          |
| SUBSTRING(字串,位置)           | 字串   | 傳回[字串]中從指定的[位置]開始到結尾的內容                              |
| SUBSTRING(字串,位置,長度)      | 字串   | 傳回[字串]中從指定的[位置]開始，到指定[長度]的內容                      |
| CONCAT(參數 [,…])              | 字串   | 傳回所有參數連接起來的字串                                              |
| CONCAT_WS(分隔字串, 參數 [,…]) | 字串   | 傳回所有參數連接起來的字串，參數之間插入指定的[分隔字串]                |
| LENGTH(字串)                   | 數字   | 傳回[字串]的長度(bytes)                                                 |
| CHAR_LENGTH(字串)              | 數字   | 傳回[字串]的長度(字元個數)                                              |
| LOCATE(字串1,字串2)            | 數字   | 傳回[字串1]在[字串2]中的位置，如果[字串2]中沒有[字串1]指定的內容就傳回0 |

## 數學

| FUNCTIONS           | RETURN | EXPLANATION                    |
|---------------------|--------|--------------------------------|
| ROUND(數字)         | 數字   | 四捨五入到整數                 |
| ROUND(數字,位數)    | 數字   | 四捨五入到指定的位數           |
| CEIL(數字)          | 數字   | 進位到整數                     |
| FLOOR(數字)         | 數字   | 捨去所有小數                   |
| TRUNCATE(數字,位數) | 數字   | 將指定的[數字]捨去指定的[位數] |
| PI()                | 數字   | 圓周率                         |
| POW(數字1,數字2)    | 數字   | [數字1]的[數字2]平方           |
| RAND()              | 數字   | 亂數                           |
| SQRT(數字)          | 數字   | [數字]的平方                   |

## 日期時間

| FUNCTION                             | RETURN    | EXPLAINATION                                       |
|--------------------------------------|-----------|----------------------------------------------------|
| CURDATE()                            | 日期      | 取得目前日期                                       |
| CURTIME()                            | 時間      | 取得目前時間                                       |
| YEAR(日期)                           | 數字      | 傳回[日期]的年                                     |
| MONTH(日期)                          | 數字      | 傳回[日期]的月                                     |
| DAY(日期)                            | 數字      | 傳回[日期]的日                                     |
| MONTHNAME(日期)                      | 字串      | 傳回[日期]的月份名稱                               |
| DAYNAME(日期)                        | 字串      | 傳回[日期]的星期名稱                               |
| DAYOFWEEK(日期)                      | 數字      | 傳回[日期]的星期，1到7的數字，表示星期日、一、二…  |
| DAYOFYEAR(日期)                      | 數字      | 傳回[日期]的日數，1到366的數字，表示一年中的第幾天 |
| QUARTER(日期)                        | 數字      | 傳回[日期]的季，1到4的數字，代表春、夏、秋、冬     |
| EXTRACT(單位 FROM 日期/時間)         | 數字      | 傳回[日期]中指定的[單位]資料                       |
| HOUR(時間)                           | 數字      | 傳回[時間]的時                                     |
| MINUTE(時間)                         | 數字      | 傳回[時間]的分                                     |
| SECOND(時間)                         | 數字      | 傳回[時間]的秒                                     |
| ADDDATE(日期, 天數)                  | 日期      | 傳回[日期]在指定[天數] 以後的日期                  |
| ADDDATE(日期, INTERVAL 數字 單位)    | 日期      | 傳回[日期]在指定[數字]的[單位]以後的日期           |
| ADDTIME(日期時間, INTERVAL數字 單位) | 日期/時間 | 傳回[日期時間]在指定[數字]的[單位]以後的日期時間   |
| SUBDATE(日期, 天數)                  | 日期      | 傳回[日期]在指定[天數] 以前的日期                  |
| SUBDATE(日期, INTERVAL 數字 單位)    | 日期/時間 | 傳回[日期]在指定[數字]的[單位]以前的日期           |
| SUBTIME(日期時間, INTERVAL數字 單位) | 日期/時間 | 傳回[日期時間]在指定[數字]的[單位]以前的日期時間   |
| DATEDIFF(日期1, 日期2)               | 數字      | 計算兩個日期差異的天數                             |

## 群組

| FUNCTION      | EXPLANATION  |
|---------------|--------------|
| MAX(運算式)   | 最大值       |
| MIN(運算式)   | 最小值       |
| SUM(運算式)   | 合計         |
| AVG(運算式)   | 平均         |
| COUNT(運算式) | 計數         |

In [79]:
cursor.close()
conn.close()